<a href="https://colab.research.google.com/github/alifzl/Zhaav-MINER-Scripts/blob/main/pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from typing import Union

In [ ]:
def createSparkSession(appname:str):
        '''
        create spark session rdd.
            Parameters:
            ---------- 
                    appname (str): The appliction name  
                    

            Returns:
                    SparkSession

        '''
        
        import pyspark
        from pyspark.sql import SparkSession
        spark = SparkSession.builder.appName(appname).getOrCreate()
        return SparkSession


*Read* CSV file
Read CSV file into DataFrame with schema and header


In [ ]:
def readcsvfile(header:str,inferSchema:str,appname:str,path:str,sep:str):
        '''
        create dataframe by reading csv file.
            Parameters:
            ---------- 
                    header(str):  auto detect header of the file  ,
                  
                    inferSchema(str):  auto detect type of columns,
                    
                    appname(str):  The appliction name,
                    
                    path(str):    file path
                    
                    sep(str): columns separator

            Returns
        --------------
               dataframe( :class:`DataFrame) : return dataframe out of csv file

        '''
        spark=createSparkSession(appname)
        dataframe=spark.read.csv(path,header=header,inferSchema=inferSchema,sep=sep) 
        return dataframe

*Read* multiply CSV file
Read CSV file into DataFrame with schema and header

In [ ]:
def multireading(mainpath:str)->list:
        '''
        create dataframe by reading csv files.

          Parameters:
          ----------                   
                    mainpath(str):  files path
                    

            Returns
        --------------
              `:class:DataFrame
          '''
        import os
        df_list = []
        for filename in os.listdir(mainpath):
          if filename.endswith('.csv'):
            filename_list = filename.split(".")
            df_name=filename_list[0]
            df=spark.read.csv(mainpath+filename,inferSchema="true",header="true")
            df.name=df_name
            df_list.append(df_name)
        return df_list   

read parquet files:
read parquet filees by pandas dataframe

In [ ]:
def readparquetfile(header,inferSchema,path:str):               
    '''
        create dataframe by reading parquet file.
        
        Parameters:
        -------------- 
                    header(str):  auto detect header of the file  ,
                  
                    inferSchema(str):  auto detect type of columns,
                    
                    appname(str):  The appliction name,
                    
                    path(str):    file path
                    

            Returns
        --------------
                :class:`DataFrame
    '''

    dataframe=spark.read.parquet(path,header=header,inferSchema=inferSchema) 
    return dataframe



read excel files:
read excel filees by pandas dataframe

In [ ]:
def readexcelfile(appname:str,path:str):
        '''
        create dataframe by reading excel file.
        
        Parameters:
        -------------- 
                   
                    appname(str):  The appliction name,
                    
                    path(str):    file path
                    

            Returns
        --------------
                :class:`DataFrame
        '''        
        import pandas as pd
        spark=createSparkSession(appname)
        dataframe= pd.read_excel(path)
        dataframe_pyspark = spark.createDataFrame(dataframe)
        return dataframe_pyspark

### writing file

In [ ]:
def writhingfile(dataframe,name:str,mode:str):
    '''
  write csv file.
  
  Parameters:
  -------------- 
          dataframe(:class:`DataFrame) : select dataframe that want to write
          name(str) : name of the file 
          mode(str) : append or write
    '''
  
  
    from py4j.java_gateway import java_import
    dataframe.write.mode(mode).csv(name+'.csv')
    java_import(spark._jvm, 'org.apache.hadoop.fs.Path')
    fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(spark._jsc.hadoopConfiguration())
    file = fs.globStatus(spark._jvm.Path(name+".csv/part*"))[0].getPath().getName()
    fs.rename(spark._jvm.Path(name+'.csv/' + file), spark._jvm.Path(name+'2.csv'))
    fs.delete(spark._jvm.Path(name+'.csv'), True)

### common command
includes show, info,types,statistic describe,name of the columns and select.

In [ ]:
def convert_npandas(dataframe,number:int=1):
    
        '''
        create pandas dataframe from spark data frame .
        
        Parameters:
        -------------- 
                   
                dataframe(:class:`DataFrame) : select dataframe that  convert to pandas dataframe
                    
                number(int):   mandatory , number of rows
                    

            Returns
        --------------
                :class:`DataFrame
        '''
        dataframe_show=dataframe.limit(number).toPandas()
        return dataframe_show


In [ ]:
def show(dataframe,number:int=None,truncate:bool=None):
        '''
        show the dataframe .
        
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe
                    
            number(int) : optional, number of rows to show.
            
            truncate : bool, optional
            If set to ``True``, truncate strings longer than 20 chars by default.
            If set to a number greater than one, truncates long strings to length ``truncate``
            and align cells right.
            
            Returns
        --------------
               """Prints the first ``n`` rows to the console.

    

        

                '''
        dataframe.show(number,truncate)

In [ ]:
def dataframetype(dataframe):
    """Returns all column names and their data types as a list
            
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe
                    
        Returns
        --------------
               Prints types of the rows """
    dataframe.dtypes

In [ ]:
def statisticdescribe(dataframe):
        """Returns statistics summary of dataframe
        
        Parameters:
        --------------
        dataframe(:class:`DataFrame) : select dataframe

        Returns
        --------------
              :class:`DataFrame"""
        dataframe_new=dataframe.describe()
        return dataframe_new

In [ ]:
def coloumes(dataframe):
    """Returns all column names  as a list
            
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe
                    
        Returns
        --------------
               Prints all column names """
    dataframe.columns

In [ ]:
def select(dataframe,coloumes:Union[list, str]):
    """select specfic columns by name and return new dataframe
            
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe
            
            coloumes(str/list): list of string coloumes names         

        Returns
        --------------
              dataframe_new(:class:`DataFrame ): return new dataframe
              
              
        Example
        -------------- 
            1- common select 
                
                dataframe_new=dataframe.select("name")


            
            
            2- also can slicing the dataframe with start and end
            
                dataframe_new=dataframe.select(dataframe.columns[start=10:end=20])"""
    dataframe_new=dataframe.select(coloumes)
    return dataframe_new
    

In [ ]:
def collect(dataframe)->list:
  """ return all of dataframe as list
            
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe
            
                   

        Returns
        --------------
              ListData(list): list of all dataframe """

  ListData=dataframe.collect()
  return ListData

## Common filtering

### starts with

In [ ]:
def startswith(dataframe,columns:str,value:str):
    """ return all row of dataframe as a new dataframe that starts with the value  
            
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe

            columns(str) : columns of dataframe
            
                   

        Returns
        --------------
              dataframe_new(:class:`DataFrame): dataframe of the value"""
    dataframe_new=dataframe.where(dataframe[columns].startswith(value))
    return dataframe_new

### endswith

In [ ]:
def endswith(dataframe,columns:str,value:str):
    """ return all row of dataframe as a new dataframe that ends with the value 
            
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe

            columns(str) : columns of dataframe
            
                   

        Returns
        --------------
              dataframe_new(:class:`DataFrame): dataframe of the value"""
    dataframe_new=dataframe.where(dataframe[columns].endswith(value))
    return dataframe_new

In [ ]:
def filt(dataframe,condition:str):
    """ return all row of dataframe as dataframe that filter by condition 
            
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe

            columns(str) : columns of dataframe
            
                   

        Returns:
        --------------
              dataframe_new(:class:`DataFrame): dataframe of the value
              
        Example:
        --------------
        dataframe_new = filt (df,df["Ward"]=="value") 

              """
        
        
    dataframe_new=dataframe.filter(condition)
    return dataframe_new

sort data

In [ ]:
def groupbydesc(dataframe,columnname:str):
  """ return  a sorted dataframe by  columnname
            
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           

            columnname(str):select the column
        Returns:
        --------------
            df_sorted (:class:`DataFrame) : new sorted dataframe 
            

            """
  df_sorted = dataframe.groupby(columnname).count().orderBy(col("count").desc())
  return df_sorted

In [ ]:
def sortascending(dataframe,coloumes:str,ascending:bool):
    """ return all sorted  dataframe with  ascending or descending method
            
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe

            columns(str) : columns of dataframe
            
            ascending(bool): if ascending equal to true it's sorted by ascending if it's equal to False sorted descending
            
                   

        Returns:
        --------------
            dataframe_asc (:class:`DataFrame) : dataframe of sort value"""

    dataframe_asc = dataframe.orderBy(coloumes,ascending=ascending) 
    return dataframe_asc

In [ ]:
def isin (dataframe,columns:str,value:str):
   """ return  a  dataframe  where the columns contain Specific  value
            
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe

            columns(str) : columns of dataframe
            
            value(str): value in columns
                   

        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : datafram contain Specific  value"""
   dataframe_new = dataframe[dataframe[columns].isin(value)]
   return dataframe_new

### Join two dataframe


In [ ]:
def join(dataframe1,dataframe2,list_coloumes:str,how:str):
    """ return  a  dataframe  combinong two dataframes
            
        Parameters:
        -------------- 
                   
            dataframe1(:class:`DataFrame) : select dataframe  number 1

            dataframe2(:class:`DataFrame) : select dataframe  number 2

            list_coloumes(str) : columns of dataframe
            
            how(str):   inner', 'outer', 'full', 'fullouter', 'full_outer', 
                        'leftouter', 'left', 'left_outer', 'rightouter', 'right', 
                        'right_outer', 'leftsemi', 'left_semi', 'semi', 'leftanti', 
                        'left_anti', 'anti', 'cross' joins
                   

        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : combinong dataframe """
    dataframe_join=dataframe1.join(dataframe2,list_coloumes,how)
    return  dataframe_join

make new coloumes

In [ ]:
def makecoloumes(dataframe,name:str,opration):
    """ return  a new  dataframe  from old dataframe by using opration
            
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           
            name(str):  name of new coloumes
            opration : opertion on some coloumes  

        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe 
            
        Example:
        --------------
        Create a new column named c1 from twice the value of the ward column . 
            dataframe_new=dataframe.withColumn("c1",2*dataframe['ward'])    
            """
    dataframe_new=dataframe.withColumn(name,opration)
    
    return dataframe_new
    

change the type of one coloume



In [ ]:
def changetype(dataframe,name_coloume:str,datatype:str):
    """ return  a new  dataframe  from old dataframe with changing data type
            
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           
            name_coloume(str):  name of new coloumes
            datatype : int,float,str 

        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe 
            

            """
    from pyspark.sql.functions import col
    dataframe_new=dataframe.withColumn(name_coloume,col = col(name_coloume).cast(datatype))
    return dataframe_new

In [ ]:
def changedate(dataframe,format:str,datecoloume):
    """ return  a new  dataframe  from old dataframe with changing data type (time )
            
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           
            datecoloume(str):  name of time coloumes
            format(str) : the main format yyyy-MM-dd HH:mm:ss. SSSS  

        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe 
            

            """
    from pyspark.sql.functions import to_date
    dataframe_new = dataframe.select(to_date(datecoloume,format))
    return dataframe_new

### concat two columns

In [ ]:
def concatcolumn(dataframe,column1:str,column2:str,aliasname:str,sep:str):
    """ return  a  dataframe  combinong two dataframes
            
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           
            column1(str):  column name 1
            column2(str):  column name 2 
            aliasname(str) : name of new column 
            sep(str) : seperator 

        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe 
            

            """
    from pyspark.sql.functions import concat_ws
    dataframe_new =dataframe.select(concat_ws(sep,dataframe[column1],dataframe[column2]).alias(aliasname))
    return dataframe_new

### new column by splite

In [ ]:
def spliting(dataframe,column:str,sep:str,aliasname:str):
  """ create  a new dataframe from spliting the old dataframe
            
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           
            column(str):  column name 
            
            aliasname(str) : name of new column 
            sep(str) : seperator 

        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe 
            

            """
  from pyspark.sql.functions import split
  dataframe_new=dataframe.withColumn(aliasname,split(dataframe[column],sep))
  return dataframe_new

### Construct a new dynamic column From two columns 


In [ ]:
def newcolumn(dataframe,newcolumnname:str,column1:str,column2:str):
    """ Construct a new dynamic column From two columns 
            
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           
            newcolumnname(str):  column name 
            
            column1(str) : name of  column 1
            column2(str) : name of  column 2 
            
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe 
            

            """
    from pyspark.sql import functions as F
    dataframe_new = dataframe.withColumn(newcolumnname, F.when((dataframe[column1].isNotNull() & dataframe[column2].isNotNull())
                                     , F.concat(dataframe[column1], dataframe[column2])).otherwise(F.lit(None)))
    return dataframe_new

### Remove columns


In [ ]:
def removecolumns(dataframe,columnsname:str):
  """ remove specific  column in dataframe 
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           
            
                       
            column(str):  column name 
            
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with remove a column
            
  """

  dataframe_new = dataframe.drop(columnsname)
  return dataframe_new

### remove other columns of dataframe if specific column doesn't have the character

In [ ]:
def removecontain(dataframe,column:str,character:str):
      
""" remove other columns of dataframe if specific column doesn't have the character
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           
            
                       
            column(str):  column name 

            character(str): specific character name
            
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with remove  columns
            
    """


    dataframe = dataframe.filter(dataframe[column].contains(character))
    return dataframe

### remove other columns of dataframe  if the column doesn't start with a specific character

In [ ]:
def removestart(dataframe,column,character:str):
          
    """ remove other columns of dataframe  if the column doesn't start with a specific character
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           
            
                       
            column(str):  column name 

            character(str): specific character name
            
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with remove  columns
            
    """

    dataframe = dataframe.filter(dataframe[column].startswith(character))
    return dataframe
    

### remove other columns of dataframe  if the column doesn't end with a specific character





In [ ]:
def removeend(dataframe,column,character:str):
              
    """ remove whole dataframe if it doesn't end with a specific character
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           
            
                       
            column(str):  column name 

            character(str): specific character name
            
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with remove  columns
            
    """
    dataframe = dataframe.filter(dataframe[column].endswith(character))
    return dataframe

### remove whole dataframe if it has null values

In [ ]:
def removenul(dataframe,column):
   
    """ remove whole dataframe if it has null values
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           
            
                       
            column(str):  column name 

            character(str): specific character name
            
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with remove  columns
            
    """
    dataframe = dataframe.filter(dataframe[column].isnull())
    return dataframe



### remove whole dataframe if it has'nt null value

In [ ]:
def removenotnul(dataframe,column):
    """ ### remove whole dataframe if it has'nt null value
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           
            
                       
            column(str):  column name 

            character(str): specific character name
            
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with remove a column
            
    """
    dataframe = dataframe.filter(dataframe[column].isnotnull())
    return dataframe

### remove dataframe if column name doesn't start with specific character with like

In [ ]:
def removelike(dataframe,column,character:str):
    """ remove whole dataframe if it doesn't contain a specific character
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           
            
                       
            column(str):  column name 

            character(str): specific character name
            
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with remove  columns
            
    """

    dataframe = dataframe.filter(dataframe[column].like(character))
    return dataframe

In [ ]:
def removeregex(dataframe,column,character:str):
    
    """ remove whole dataframe if it doesn't contain a specific character with regex
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           
            
                       
            column(str):  column name 

            character(str): specific character name
            
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with remove  columns
            
    """
    
    dataframe = dataframe.filter(dataframe[column].rlike(character))
    return dataframe

### writing Query

In [ ]:
def createQuery(dataframe,name:str,Query:str):
   """ Construct a new datafram(table) with sql Query
            
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           
            name(str):   table name
            Query(str): sql Query

        Returns:
        --------------
            sparksql (:class:`DataFrame) : new dataframe



          Examples
        --------
        >>> df.createOrReplaceTempView("people")
        >>> df2 = df.filter(df.age > 3)
        >>> df2.createOrReplaceTempView("people")
        >>> df3 = spark.sql("select * from people")
        >>> sorted(df3.collect()) == sorted(df2.collect())
        True
        >>> spark.catalog.dropTempView("people")
        """
   dataframe.createOrReplaceTempView(name)
   sparksql=spark.sql(Query)
   return sparksql

### create a object by Query 

In [ ]:
def Querytransfor(dataframe,Query:str):
    """ Construct a new datafram with sql Query
            
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           
            Query(str): sql Query

        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe




    """
    from pyspark.ml.feature import SQLTransformer
    sqltrans=SQLTransformer(statement=Query+"from __THIS__")
    dataframe_new = sqltrans.transform(dataframe)
    return dataframe_new

### null counts

In [ ]:
def null_value_calc(dataframe)->list:
  """ return  a  list of null value
            
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           

        Returns:
        --------------
            (null_columns_counts) (list) : list of null value
            

            """
  from pyspark.sql.functions import col
  null_columns_counts=[]
  numrows=dataframe.count()
  for k in dataframe.columns:
    nullrows=dataframe.where(col(k).isNull()).count()
    if (nullrows>0):
      temp = k,nullrows,(nullrows/numrows)*100
      null_columns_counts.append(temp)
  return (null_columns_counts)

#### Replace all nulls with a specific value


In [ ]:
def replacenull(dataframe,columns:str,newvalue:str):
    """ replace all missing data with optional value 
            
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           
            
            newvalue(str) : optional value

            columns(str) : name of  column
            
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with missing replace 
            

        """
    dataframe = dataframe.fillna({columns: newvalue})
    return dataframe

#### fill all nulls with a specific value


In [ ]:
def fillnull(dataframe,newvalue:Union[str,int],subset:list):
  """ replace all missing data with optional value for entire dataframe
            
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           
            
            newvalue(str,int) : optional value

            columns(str) : name of  column
            
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with missing replace 
            

        """
  dataframe_new=dataframe.na.fill(newvalue,subset)
  return dataframe_new

#### drop null values
How and Thresh and Subset is optional

In [ ]:
def dropnnull(dataframe,thresh:int=None,how:str="any",subset:list=None):
  """ drop all missing data column in dataframe
            
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           
            
            thresh(int,optional) : optional value

            how(str,optional) : include (any, all)
            
            subset(list,optional): list of the column
            
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with drop column 
            
  """
  dataframe_new=dataframe.na.drop(how,thresh=thresh,subset=subset)
  return dataframe_new

##### drop null by filter method

In [ ]:
def dropnnullbyfilter(dataframe,column:str):
    """ drop all missing data column in dataframe using filter method
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           

            
            column(str):  column name 
            
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with drop column 
            
  """
    dataframe_new=dataframe.filter(dataframe[column].isNotNull())
    return dataframe_new

#### Drop duplicate rows in a dataframe (distinct) 
columnsname is optional

In [ ]:
def dropDuplicates(dataframe,columnsname:list):
  """ drop all duplicate  column in dataframe 
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           

            
            column(list,otional):  column name 
            
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with drop the duplicate data 
            
  """
  dataframe_new = dataframe.drop_duplicates(subset=columnsname)
  return dataframe_new

#### Replace specific value with null 


In [ ]:
def replacewithnull(dataframe,specificvalue:Union[int,str],columnsname:str):
    
  """ replace specific value  column in dataframe with null value
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           
            
            specificvalue(int,str) : specific value 
            
            column(list,otional):  column name 
            
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with specific value
            
  """
  dataframe_new = dataframe.replace({specificvalue: None}, subset=[columnsname])
  return dataframe_new

## impute null data

### impute null data by avarage strategy

In [ ]:
def imputnullavg(dataframe,inputCols:list,outputCols:list):   
    """ return a dataframe  with imputing null value with average 
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe        
            
            inputCols(list) : name of input columns 

            outputCols(list) : name of output columns 

            
            
                       
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe   with imputing null value with average 
            
            """
    from pyspark.ml.feature import Imputer
    a= Imputer(inputCols=inputCols,outputCols=outputCols)
    a.setStrategy("mean")
    dataframe = a.fit(dataframe).transform(dataframe)
    return dataframe

### impute null data by median strategy


In [ ]:
def imputnullmedian(dataframe,inputCols:list,outputCols:list):   
    """ return a dataframe  with imputing null value with median 
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe        
            
            inputCols(str) : name of input columns 

            outputCols(str) : name of output columns 

            
            
                       
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe   with imputing null value with median 
            
            """
    from pyspark.ml.feature import Imputer
    a= Imputer(inputCols=inputCols,outputCols=outputCols)
    a.setStrategy("median")
    dataframe=a.fit(dataframe).transform(dataframe)
    return dataframe

### find basic statistic parameter to replace with null values

In [ ]:
def findstatistict(dataframe,column:str)->dict:
    """ return basic statistic parameter 
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           
            
                       
            column(str):  column name 
            
        Returns:
        --------------
            avg(int): average of the column

            stddev(int): standard deviation of the column

            min(int): minimum of the column

            max(int): maximum of the column

            q1(int): quartile first of the column

            q2(int): quartile second of the column

            q3(int): quartile third of the column
            
    """





    avg=eval(dataframe.summary().select("summary",column).collect()[1][1])
    stddev=eval(dataframe.summary().select("summary",column).collect()[2][1])
    min=eval(dataframe.summary().select("summary",column).collect()[3][1])
    max=eval(dataframe.summary().select("summary",column).collect()[7][1])
    q1=eval(dataframe.summary().select("summary",column).collect()[4][1])
    q2=eval(dataframe.summary().select("summary",column).collect()[5][1])
    q3=eval(dataframe.summary().select("summary",column).collect()[6][1])

    base={"avg":avg,"stddev":stddev,"min":min,"max":max,"quartile1":q1,"quartile2":q2,"quartile3":q3}
    return base


## String Operations

In [ ]:
def wordsonly(dataframe,columnname:str):
    """ return dataframe with only words and space between.
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           
            
                       
            columnname(str):  column name 

  

            
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with trim a column
            
    """ 

    dataframe_new = dataframe.withColumn(columnname,regexp_replace(col(columnname), '[^A-Za-z]+', ' '))
    return dataframe_new

### remove dataframe if column  doesn't have a specefic list name


In [ ]:
def removelist(dataframe,column,character:Union[tuple,str]):
    """ remove whole dataframe if it doesn't contain a specific character with regex
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           
            
                       
            column(str):  column name 

            character(tuple,str): specific character name
            
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with remove  columns
            
    """
    dataframe = dataframe.filter(dataframe[column].isin(character))
    return dataframe

### String Functions


#### substring the dataframe

In [ ]:
def substrcolumn(dataframe,column:str,newcolumn:str,start:int,lenght:int):
    """ sub string a column and make new column
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           
            
                       
            column(str):  column name 

            newcolumn(str): new column name 

            start (int): starting ponit
            
            lenght(int) : lenght of column 
            
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with substring a column
            
    """
    dataframe_new = dataframe.withColumn(newcolumn, dataframe[column].substr(start, lenght))
    return dataframe_new

#### Trim the spaces from both ends for the specified string column.

In [ ]:
def trimstrcolumn(dataframe,column,newcolumn:str):
    """ Trim the spaces from both ends for the specified string column.
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           
            
                       
            column(str):  column name 

            newcolumn(str): new column name 


            
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with trim a column
            
    """

    from pyspark.sql import functions as F
    dataframe = dataframe.withColumn(newcolumn, F.trim(column))
    return dataframe

### left trim and right trim

In [ ]:
def ltrim(dataframe,newcolumn:str,column:str):
    """ left Trim the spaces from string column.
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           
            
                       
            column(str):  column name 

            newcolumn(str): new column name 


            
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with trim a column
            
    """
    from pyspark.sql import functions as F
    dataframe_new = dataframe.withColumn(newcolumn, F.ltrim(dataframe[column]))
    return dataframe_new

In [ ]:
def rtrim(dataframe,newcolumn:str,column:str):
    """ right Trim the spaces from string column.
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           
            
                       
            column(str):  column name 

            newcolumn(str): new column name 


            
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with trim a column
    """        
    from pyspark.sql import functions as F
    dataframe_new = dataframe.withColumn(newcolumn, F.rtrim(dataframe[column]))
    return dataframe_new

#### Left-pad the string column to width len with pad

In [ ]:
def leftpad(dataframe,column,newcolumn:str,character:str,digitnumber:int):
    """ Left-pad the string column to width len with pad.
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           
            
                       
            column(str):  column name 

            newcolumn(str): new column name 

            character(str): a character to add a column

            digitnumber(int): number of character

            
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with trim a column
            
    """

    from pyspark.sql import functions as F
    dataframe_new = dataframe.withColumn(newcolumn, F.lpad(dataframe[column], digitnumber, character))
    return dataframe_new

### Concatenate with character between

In [ ]:
def concattwo(dataframe,newcolumn:str,column1:str,column2:str,chrbetween:str):
    """ Concatenate with character between.
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           
            
            newcolumn(str): new column name 
            
            column1(str):  first column name 
            
            column2(str):  secend column name

            chrbetween(str): a character to add between a column


            
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with  with a character between.
            """
    from pyspark.sql import functions as F
    dataframe_new = dataframe.withColumn(newcolumn, F.concat(dataframe[column1], F.lit(chrbetween), dataframe[column2]))
    return dataframe_new


### Concatenate with Separator


In [ ]:
def concatwithesep(dataframe,newcolumn:str,*cols:str,chrbetween:str):
        from pyspark.sql import functions as F
        """"Returns a new :class:`DataFrame` by adding a column or replacing the
        existing column that has the same name.

        The column expression must be an expression over this :class:`DataFrame`; attempting to add
        a column from some other :class:`DataFrame` will raise an error.

        Parameters
        ----------
        colName : str
            string, name of the new column.
        col : :class:`Column`
            a :class:`Column` expression for the new column.

        Notes
        -----
        This method introduces a projection internally. Therefore, calling it multiple
        times, for instance, via loops in order to add multiple columns can generate big
        plans which can cause performance issues and even `StackOverflowException`.
        To avoid this, use :func:`select` with the multiple columns at once.
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with trim a column
            """
       
        dataframe = dataframe.withColumn(newcolumn, F.concat_ws(chrbetween, *cols))
        return dataframe


## Number Operations


### Round the number

In [ ]:
def roundnumber(dataframe,newcolumn:str,column:str,scale:int):
    """ return round of integer columns
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           
            
            newcolumn(str): new column name 
            
            column(str):   column name

            scale(int): a scale of rounding


            
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with  with a round of a columns
            """
    from pyspark.sql import functions as F
    dataframe_new = dataframe.withColumn(newcolumn, F.round(dataframe[column], scale))
    return dataframe_new

### Floor of the number

In [ ]:
def floarnumber(dataframe,newcolumn:str,column:str):
    """ return floor of integer columns
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           
            
            newcolumn(str): new column name 
            
            column(str):   column name

            scale(int): a scale of rounding


            
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with  with a round of a columns
            """
    from pyspark.sql import functions as F
    dataframe_new = dataframe.withColumn(newcolumn, F.floor(dataframe[column]))
    return dataframe_new

### Ceiling 

In [ ]:
def Ceilingnumber(dataframe,newcolumn:str,column:str):
    """ return Ceilin of integer columns
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           
            
            newcolumn(str): new column name 
            
            column(str):   column name

          


            
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with Ceilin a columns
            """
    from pyspark.sql import functions as F
    dataframe_new = dataframe.withColumn(newcolumn, F.ceil(dataframe[column]))
    return dataframe_new

### Absolute Value

In [ ]:
def Absolutenumber(dataframe,newcolumn:str,column:str):
    """ return absolute  of integer columns
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           
            
            newcolumn(str): new column name 
            
            column(str):   column name

          


            
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with absolute a columns
            """
    from pyspark.sql import functions as F
    dataframe_new = dataframe.withColumn(newcolumn, F.abs(dataframe[column]))
    return dataframe_new


### raised to power

In [ ]:
def powernumber(dataframe,newcolumn:str,column1:str,column2:str):
    """ return a column calculate first column power by second column 
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           
            
            newcolumn(str): new column name 
            
            column1(str): first  column name

            column2(str): second  column name

          


            
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with a column calculate first column power by second column
            """
    from pyspark.sql import functions as F
    dataframe = dataframe.withColumn(newcolumn, F.pow(column1,column2))
    return dataframe


### Select smallest value out of multiple columns

In [ ]:
def smallestcolumns( dataframe,newcolumn:str,cols:tuple):
    """ return a  smallest value out of multiple columns 
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           
            
            newcolumn(str): new column name 
            
            cols(tuple):   name of the columns

          


            
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with a  smallest value out of multiple columns 
            """
    a=str(cols)
    from pyspark.sql import functions as F
    dataframe_new = dataframe.withColumn(newcolumn, eval("F.least"+a))
    return dataframe_new

### Select largest value out of multiple columns

In [ ]:
def greatestcolumns( dataframe,newcolumn:str,cols:str):
    """ return a  greatest value out of multiple columns 
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           
            
            newcolumn(str): new column name 
            
            cols(tuple):   name of the columns

          


            
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with a  greatest value out of multiple columns
            """
    from pyspark.sql import functions as F
    dataframe = dataframe.withColumn(newcolumn, eval("F.greatest"+str(cols)))
    return dataframe
                                     

### Date & Timestamp Operations


#### Convert a string of known format to a date (excludes time information)

In [ ]:
def formattdate(dataframe,newcolomn:str,column:str,formattime:str):
    """ Convert a string of known format to a date (excludes time information)
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           
            
            newcolumn(str): new column name 
            
            column(str):   name of the columns

           formattime(str): standard date format (yyyy-MM-dd HH:mm:ss)
          


            
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with a string of known format to a date (excludes time information)
            """
    from pyspark.sql import functions as F
    dataframe_new = dataframe.withColumn(newcolomn, F.to_date(column, formattime))
    return dataframe_new


#### Convert a string of known format to a timestamp

In [ ]:
def formatotimestamp(dataframe,newcolumn:str,colomn:str,formattime:str):
    """ Convert a string of known format to a timestamp 
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           
            
            newcolumn(str): new column name 
            
            column(str):   name of the columns

           formattime(str): standard date format (yyyy-MM-dd HH:mm:ss.SSS)
          


            
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with a column calculate first column power by second column
            """
    from pyspark.sql import functions as F
    dataframe = dataframe.withColumn(newcolumn, F.to_timestamp(colomn, formattime))
    return dataframe


#### Get second/minute/hour/day/month/year from date:       

In [ ]:
def creatyear(dataframe,datecoloums,newcolumn):
    
    """ return a column as a year taken from the date column 
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe  
            
            datecoloums(str):   name of the columns         
            
            newcolumn(str): new column name 
            
                       
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with a  year from date column
            """    
    from pyspark.sql.functions import year
    datafram_new=dataframe.select(year(datecoloums).alias(newcolumn))
    return datafram_new

In [ ]:
def creatmonth(dataframe,datecoloums,newcolumn):
        
    """ return a column as a month taken from the date column 
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe  
            
            datecoloums(str):   name of the columns         
            
            newcolumn(str): new column name 
            
                       
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with a  month from date column
            """    
    from pyspark.sql.functions import year
    from pyspark.sql.functions import month
    datafram_new=dataframe.select(month(datecoloums).alias(newcolumn))
    return datafram_new

In [ ]:
def creatdaymonth(dataframe,datecoloums,newcolumn):
        
    """ return a column as a day of month taken from the date column 
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe  
            
            datecoloums(str):   name of the columns         
            
            newcolumn(str): new column name 
            
                       
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with a  day of month from date column
            """    

    from pyspark.sql.functions import dayofmonth
    datafram_new=dataframe.select(dayofmonth(datecoloums).alias(newcolumn))
    return datafram_new

In [ ]:
def createhour(dataframe,datecoloums,newcolumn):
    """ return a column as hour taken from the date column 
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe  
            
            datecoloums(str):   name of the columns         
            
            newcolumn(str): new column name 
            
                       
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with a  hour from date column
            """    
    from pyspark.sql.functions import hour
    datafram_new=dataframe.select(hour(datecoloums).alias(newcolumn))
    return datafram_new

In [ ]:
def createminute(dataframe,datecoloums,newcolumn):
    """ return a column as minute taken from the date column 
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe  
            
            datecoloums(str):   name of the columns         
            
            newcolumn(str): new column name 
            
                       
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with a  minute from date column
            """    
    from pyspark.sql.functions import minute
    datafram_new=dataframe.select(minute(datecoloums).alias(newcolumn))
    return datafram_new

In [ ]:
def createsecond(dataframe,datecoloums,newcolumn):
    """ return a column as second taken from the date column 
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe  
            
            datecoloums(str):   name of the columns         
            
            newcolumn(str): new column name 
            
                       
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with a  second from date column
            """
    from pyspark.sql.functions import second
    datafram_new=dataframe.select(second(datecoloums).alias(newcolumn))
    return datafram_new

#### Add & subtract days


In [ ]:
def adddatedays(dataframe,newcolumn:str,column,dateadded:int):
    """ add days to date column
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe  
            
            datecoloums(str):   name of the columns         
            
            newcolumn(str): new column name 
            
            dateadded(int) : number of day add to date column
            
                       
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with a new column that add days to date column
            """
    from pyspark.sql import functions as F
    dataframe_new = dataframe.withColumn(newcolumn, F.date_add(column, dateadded))
    return dataframe_new

In [ ]:
def subdatedays(dataframe,newcolumn:str,column,datesub:int):
    """ subtract days from date column
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe  
            
            datecoloums(str):   name of the columns         
            
            newcolumn(str): new column name 
            
            datesub(int) : number of day subtract from date column
            
                       
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with a new column that subtract days from date column
            """
    from pyspark.sql import functions as F
    datafram_new = dataframe.withColumn(newcolumn, F.date_sub(column, datesub))
    return datafram_new

#### Add & Subtract months


In [ ]:
def addmonth(dataframe,newcolumn,column,dateadded):
    """ add month to date column
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe  
            
            datecoloums(str):   name of the columns         
            
            newcolumn(str): new column name 
            
            dateadded(int) : number of month subtract from date column
            
                       
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with a new column that add month from date column
            """
    from pyspark.sql import functions as F 
    dataframe_new = dataframe.withColumn(newcolumn, F.add_months(column, dateadded))
    return dataframe_new

#### Get number of days between two dates

In [ ]:
def daybetween(dataframe,newcolumn,column1,column2):
    """ return a dataframe with diffrent two date column 
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe        
            
            newcolumn(str): new column name 
            
            column1(str) : number of month subtract from date column

            column2(str) : number of month subtract from date column
            
                       
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with diffrent two date column 
            """
    from pyspark.sql import functions as F
    dataframe_new = dataframe.withColumn(newcolumn, F.datediff(column1, column2))
    return dataframe_new


#### Get number of months between two dates


In [ ]:
def monthbetween(dataframe,newcolumn,column1,column2):
    """ return a dataframe with diffrent two date column 
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe        
            
            newcolumn(str): new column name 
            
            column1(str) : number of month subtract from date column

            column2(str) : number of month subtract from date column
            
                       
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with diffrent two date column 
            """
    from pyspark.sql import functions as F
    dataframe_new = dataframe.withColumn(newcolumn, F.months_between(column1, column2))
    return dataframe_new

### Array Operations

In [ ]:
def creatarray(dataframe,newcolumn:str,cols:tuple):
    """ create array from selected columns
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe        
            
            newcolumn(str): new column name 
            
            cols(tuple(str)) : name of columns

            
            
                       
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with array column from selected columns
            """
    
    from pyspark.sql import functions as F
    dataframe_new = dataframe.withColumn(newcolumn, eval("F.array"+str(cols)))
    return dataframe_new


#### Empty Array


In [ ]:
def creatarray(dataframe,newcolumn):
    """ create  empty array 
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe        
            
            newcolumn(str): new column name 
            

            
            
                       
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with a empty array 
            """
    
    from pyspark.sql import functions as F
    dataframe_new = dataframe.withColumn(newcolumn, F.array([]))
    return dataframe_new

#### Array Size/Length

In [ ]:
def sizearray(dataframe,newcolumn:str,columnarray:str):
    """ return size of array
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe        
            
            newcolumn(str): new column name 

            columnarray(str) : name of column array
            

            
            
                       
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe  
            """
    from pyspark.sql import functions as F
    dataframe_new = dataframe.withColumn(newcolumn, F.size(F.col(columnarray)))
    return dataframe_new

## Aggregation Operations

#### Row Count       

In [ ]:
def valuecount(dataframe,columname:str)->int:
    """ return count of selected column
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe        
            


            columname(str) : name of column 
            

            
            
                       
        Returns:
        --------------
            count (int) :  count of selected column 
            """
    count=dataframe.select(columname).count()
    return count

In [ ]:
def meancolumn(dataframe,groupbycolumns:tuple,meancolumns:str):
    """ return mean of selected column
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe        
            
            groupbycolumns(tuple) : name of columns 

            meancolumns(str):mean column
            

            
            
                       
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with mean of selected column
            
            """

    dataframe_new=dataframe.groupBy(groupbycolumns).mean(meancolumns)
    return dataframe_new

In [ ]:
def maxncloumn(dataframe,groupbycolumns:tuple,maxcolumns:str):
    """ return max of selected column
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe        
            
            groupbycolumns(tuple) : name of columns 

            maxcolumns(str):mean column
            

            
            
                       
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with max of selected column
            
            """
    dataframe_new=dataframe.groupBy(groupbycolumns).max(maxcolumns)
    return dataframe_new

In [ ]:
def mincloumn(dataframe,groupbycolumns:tuple,mincloumns:str):
    """ return min of selected column
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe        
            
            groupbycolumns(tuple) : name of columns 

            mincloumns(str):mean column
            

            
            
                       
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with min of selected column
            
            """
    dataframe_new=dataframe.groupBy(groupbycolumns).min(mincloumns)
    return dataframe_new

### Advanced Operations


### PySpark Repartition provides a full shuffling of data

In [ ]:
def repartition(dataframe,column:str):
    """ return dataframe  with a new repartition
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe        
            
            column(str) : name of columns 

          

            
            
                       
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe  with a new repartition
            
            """

    dataframe_new = dataframe.repartition(dataframe[column])
    return dataframe_new


In [ ]:
def casewhen(casewhenexp:str,dataframe): 
  """ return dataframe with only words and space between.
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           
            
                       
            casewhenexp(str):  case when expression

  

            
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new dataframe with operation on column

        Example:
        --------------

        dataframe_new = dataframe.withColumn("mood",expr("CASE WHEN EST2 in('4','5') THEN 'chill' WHEN EST2 in('1','2') THEN 'highstrung' ELSE 'neutral' END AS mood"))

            
    """  
  from pyspark.sql.functions import expr

  dataframe_new = dataframe.withColumn("vert",expr(casewhenexp))
  return dataframe_new

# machine learning

### Data Prepration

In [ ]:
def dependent_var(dataframe,dependent_var):
  """ return  a  dataframe with numeric dependent variable 
            
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           

            columnname(str):encoding the selected dependent column
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new  dataframe with numeric dependent variable 
      """
  from pyspark.sql.types import StringType 
  renamed = dataframe.withColumnRenamed(dependent_var,'label')
  # Make sure dependent variable is numeric and change if it's not
  if str(renamed.schema['label'].dataType) != 'IntegerType':
      renamed = renamed.withColumn("label", renamed["label"].cast(FloatType()))
  return renamed

In [ ]:
def dummyencoding(dataframe,column:str): 
  """ return  a  dataframe with encoding the selected column
            
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           

            columnname(str):encoding the selected column
        Returns:
        --------------
            dataframe_new (:class:`DataFrame) : new encoding dataframe 
            

  """
  import pyspark.sql.functions as F 
  categ = dataframe.select(column).distinct().rdd.flatMap(lambda x:x).collect()
  exprs = [F.when(F.col(column) == cat,1).otherwise(0)\
              .alias(str(cat)) for cat in categ]
  dataframe_new = dataframe.select(dataframe.columns+exprs).drop(column)
  return dataframe_new

In [ ]:
def MLClassifierDFPrep(dataframe,input_columns:list,dependent_var:str,treat_outliers:bool=True,treat_neg_values:bool=True):
    """ return a rescaling dataframe with data prepration ( checking skewness and outliers)
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe        
            
            inputCols(str) : name of input columns 

            dependent_var(str) : name of dependent columns 

            treat_outliers(bool) : modify dataframe for outliers
            
            treat_neg_values(bool) : modify dataframe for negative values
            
            
                       
        Returns:
        --------------
            final_data (:class:`DataFrame) : the rescaling dataframe  with data prepration 
              
            
            """  

    from pyspark.ml.feature import VectorAssembler, StringIndexer, MinMaxScaler
    from pyspark.sql.types import StringType 
    from pyspark.sql.functions import skewness,log,when,array,array_min,exp
    renamed = df.withColumn("label_str", df[dependent_var].cast(StringType())) 
    indexer = StringIndexer(inputCol="label_str", outputCol="label") 
    indexed = indexer.fit(renamed).transform(renamed)

    numeric_inputs = []
    string_inputs = []
    for column in input_columns:
        if str(indexed.schema[column].dataType) == 'StringType':
            indexer = StringIndexer(inputCol=column, outputCol=column+"_num") 
            indexed = indexer.fit(indexed).transform(indexed)
            new_col_name = column+"_num"
            string_inputs.append(new_col_name)
        else:
            numeric_inputs.append(column)
            
    if treat_outliers == True:
        print("We are correcting for non normality now!")
        d = {}
        for col in numeric_inputs: 
            d[col] = indexed.approxQuantile(col,[0.01,0.99],0.25) 
        for col in numeric_inputs:
            skew = indexed.agg(skewness(indexed[col])).collect() 
            skew = skew[0][0]
            if skew > 1:
                indexed = indexed.withColumn(col, \
                log(when(df[col] < d[col][0],d[col][0])\
                .when(indexed[col] > d[col][1], d[col][1])\
                .otherwise(indexed[col] ) +1).alias(col))
                print(col+" has been treated for positive (right) skewness. (skew =)",skew,")")
            elif skew < -1:
                indexed = indexed.withColumn(col, \
                exp(when(df[col] < d[col][0],d[col][0])\
                .when(indexed[col] > d[col][1], d[col][1])\
                .otherwise(indexed[col] )).alias(col))
                print(col+" has been treated for negative (left) skewness. (skew =",skew,")")

            
    minimums = df.select([min(c).alias(c) for c in df.columns if c in numeric_inputs]) 
    min_array = minimums.select(array(numeric_inputs).alias("mins")) 
    df_minimum = min_array.select(array_min(min_array.mins)).collect() 
    df_minimum = df_minimum[0][0] 

    features_list = numeric_inputs + string_inputs
    assembler = VectorAssembler(inputCols=features_list,outputCol='features')
    output = assembler.transform(indexed).select('features','label')

    if df_minimum < 0:
        print(" ")
        print("WARNING: The Naive Bayes Classifier will not be able to process your dataframe as it contains negative values")
        print(" ")
    
    if treat_neg_values == True:
        print("You have opted to correct that by rescaling all your features to a range of 0 to 1")
        print(" ")
        print("We are rescaling you dataframe....")
        scaler = MinMaxScaler(inputCol="features", outputCol="scaledFeatures")

        scalerModel = scaler.fit(output)

        scaled_data = scalerModel.transform(output)
        final_data = scaled_data.select('label','scaledFeatures')
        final_data = final_data.withColumnRenamed('scaledFeatures','features')
        print("Done!")

    else:
        print("You have opted not to correct that therefore you will not be able to use to Naive Bayes classifier")
        print("We will return the dataframe unscaled.")
        final_data = output
    
    return final_data

In [ ]:
def mainclustring(dataframe,inputCols:list):
    """ return a vectorize dataframe 
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe        
            
            inputCols(str) : name of input columns 

            
            
                       
        Returns:
        --------------
            dataframe_kmeans (:class:`DataFrame) : the vectorize dataframe  
              
            
            """  

  from pyspark.ml.feature import VectorAssembler
  vecAssembler = VectorAssembler(inputCols=input_columns, outputCol="features")
  dataframe_kmeans = vecAssembler.transform(dataframe)
  return dataframe_kmeans

### min/max scaler & Standard Scaler

In [ ]:
def StandardScalerclustring(dataframe,inputCol:list):
    """ return a scaling dataframe 
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe        
            
            inputCols(str) : name of input columns 

            
            
                       
        Returns:
        --------------
            scale_df (:class:`DataFrame) : the scaling dataframe  
              
            
            """  
    from pyspark.ml.feature import StandardScaler
    tr=mainclustring(dataframe,inputCol)
    scaler = StandardScaler(inputCol="features",outputCol="scaled",withStd=True,withMean=False)
    scale_df=scaler.fit(tr).transform(tr)
    scale_df=scale_df.drop("features")
    scale_df=scale_df.withColumnRenamed("scaled","features")
    return scale_df

In [ ]:
def minmaxscaling(dataframe,input_columns): 
  """ return a scaling dataframe 
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe        
            
            inputCols(str) : name of input columns 

            
            
                       
        Returns:
        --------------
            scale_df (:class:`DataFrame) : the scaling dataframe  
              
            
            """  
  from pyspark.ml.feature import MinMaxScaler
  tr=mainclustring(dataframe,input_columns)
  scaler = MinMaxScaler(inputCol="features",outputCol="scaled")
  scale_df=scaler.fit(tr).transform(tr)
  scale_df=scale_df.drop("features")
  scale_df=scale_df.withColumnRenamed("scaled","features")
  return scale_df


### clustering with kmeans 

In [ ]:
def keamnsclustering(kmeansmethod:str,scaleingmethod:str,dataframe,inputCols:list,k:int,seed:int,itr:int)->list:
    """ return  silhouette,center of kmeans
             
        Parameters:
        --------------
            
            kmeansmethod(str) : selecct   KMeans or BisectingKMeans method
                
            scaleingmethod(str): selecct   MinMaxScaler or StandardScaler method

            dataframe(:class:`DataFrame) : select dataframe        
            
            inputCols(str) : name of input columns 

            k(int) : number of center 
            
            seed(int) : seed of model

            itr(int) : number of iteration of model
            
            
                       
        Returns:
        --------------
            silhcent (list) : return a tuple with silhouette,center of kmeans ,prediction dataframe 
              
            
           """ 
    silhcent=list()       
    from pyspark.ml.clustering import BisectingKMeans
    import numpy as np
    from pyspark.ml.feature import StandardScaler
    from pyspark.ml.clustering import KMeans
    from pyspark.ml.evaluation import ClusteringEvaluator
    if scaleingmethod=="MinMaxScaler":
      tr=minmaxscaling(dataframe,inputCols)
    elif scaleingmethod=="StandardScaler":
      tr=StandardScalerclustring(dataframe,inputCols)
    for k in np.arange(2,k+1).tolist():
      if kmeansmethod == "KMeans": 
        Means=KMeans()
      elif kmeansmethod=="BisectingKMeans":
        Means=BisectingKMeans()
      Means.setMaxIter(itr)
      Means.setK(k)
      Means.setSeed(seed)
      Model = Means.fit(tr)
      pred = Model.transform(tr)
      Evaluator = ClusteringEvaluator()
      silhouette=Evaluator.evaluate(pred)
      center = Model.clusterCenters()
      s=(silhouette,center,pred)
      silhcent.append(s)
    return silhcent

### predictions of kmeans method 

In [ ]:
def predictions(kmeansmethod:str,scaleingmethod:str,dataframe,inputCols:list,k:int,seed:int,itr:int,numberbest:int):  
  """ return  perdiction dataframe
             
        Parameters:
        --------------
            
            kmeansmethod(str) : selecct   KMeans or BisectingKMeans method
                
            scaleingmethod(str): selecct   MinMaxScaler or StandardScaler method

            dataframe(:class:`DataFrame) : select dataframe        
            
            inputCols(str) : name of input columns 

            k(int) : number of center 
            
            seed(int) : seed of model

            itr(int) : number of iteration of model

            numberbest(int): number of best silhouette kmeans  calculated
            
            
                       
        Returns:
        --------------
            scale_df (:class:`DataFrame) : the predictions dataframe  
              
            
           """ 
  predictions=keamnsclustering(kmeansmethod,scaleingmethod,dataframe,inputCols,k,seed,itr)[numberbest-2][2]
  df_predictions=predictions.groupBy("prediction").agg(min(predictions.BALANCE).alias("Min BALANCE"),max(predictions.BALANCE).alias("Max BALANCE"))
  return df_predictions


### LDA for nlp clusstering 

In [ ]:
def regremvec(inputCols:list,dataframe)->tuple: 
  """ regex tokenizer and vector dataframe.
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           
            
                       
             inputCol(list):  column name 

             


            
        Returns:
        --------------
            regremvec (tuple) : tuple with three main dataframe(raw_words,df_vect,cvmodel)
            
    """
  # Tokenize
  raw_words=dataframe
  for count, inputCol in enumerate(inputCols):
    inputCol=inputCol
    outputCol="words"+str(count+1)
    regex_tokenizer = RegexTokenizer(inputCol=inputCol, outputCol=outputCol, pattern="\\W")
    raw_words = regex_tokenizer.transform(raw_words)
 # Remove Stop words
  words_df=raw_words
  for count, inputCol in enumerate(inputCols):
    inputCol="words"+str(count+1)
    outputCol="filtered"+str(count+1)
  
    remover = StopWordsRemover(inputCol="words", outputCol="filtered")
    words_df = remover.transform(words_df)

  df_vect=words_df
  for count, inputCol in enumerate(inputCols):
    inputCol="filtered"+str(count+1)
    outputCol="features"+str(count+1)    
    cv = CountVectorizer(inputCol="filtered", outputCol="features")
    cvmodel = cv.fit(words_df)
    df_vect = cvmodel.transform(df_vect)
    regremvec=(raw_words,df_vect,cvmodel)
  return regremvec

  # Zero Index Label Column


In [ ]:
def LDAplot(inputCol:list,dataframe,kmax:int):
  """ LDA datframe with plot
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           
            
                       
            inputCol(list):  column name 

            
            kmax(int): maximum k of LDA


            
        Returns:
        --------------
            lda (:class:`DataFrame) : lda datframe
            """
  import matplotlib.pyplot as plt
  from mpl_toolkits.mplot3d import Axes3D
  import numpy as np
  df_vect=regremvec(inputCol,dataframe)[1]
  ll = np.zeros(kmax)
  lp = np.zeros(kmax)
  for k in np.arange(2,kmax).tolist():
      lda = LDA(k=k, maxIter=10)
      model = lda.fit(df_vect)
      ll[k] = model.logLikelihood(df_vect)
      lp[k] = model.logPerplexity(df_vect)
      
  fig, ax = plt.subplots(1,1, figsize =(8,6))
  ax.plot(np.arange(2,kmax).tolist(),ll[2:kmax])
  ax.set_xlabel('k')
  ax.set_ylabel('ll')

  fig, ax = plt.subplots(1,1, figsize =(8,6))
  ax.plot(np.arange(2,kmax).tolist(),lp[2:kmax])
  ax.set_xlabel('k')
  ax.set_ylabel('lp')
  return lda

In [ ]:
def topic(inputCol:list,dataframe,k:int,maxIter:int):  
  """ return  a topic dataframe 
            
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe           

            inputCol(str):input column list
        Returns:
        --------------
            results (:class:`DataFrame) : topic dataframe 
            

            """
  # Make predictions
  # Trains a LDA model.
  lda = LDA(k=k, maxIter=maxIter)
  df_vect=regremvec(inputCol,dataframe)[0]
  model = lda.fit(df_vect)
  transformed = model.transform(df_vect)
  transformed.toPandas()

  # Convert topicdistribution col from vector to array
  # We need to create a udf for this one
  to_array = udf(lambda x: x.toArray().tolist(), ArrayType(DoubleType()))
  recommendations = transformed.withColumn('array', to_array('topicDistribution'))

  # Find the best topic value that we will call "max"
  max_vals = recommendations.withColumn("max",array_max("array"))

  # Find the index of the max value found above which translates to our topic!
  argmaxUdf = udf(lambda x,y: [i for i, e in enumerate(x) if e==y ])
  results = max_vals.withColumn('topic', argmaxUdf(max_vals.array,max_vals.max))
  return results

### Gaussian Mixture Modeling

In [ ]:
def Gaussianplot(dataframe,input_columns:list,kmax:int,maxIter:int,seed:int):
  """ return a new vectorize dataframe  with plot of k
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe  

            input_columns(list) :input columns     
            
            kmax(int) : maximum of k 

            maxIter(int) : maximum iteration 

            seed(int) : seed of Gaussian Mixture cluss cluster model

            
            
                       
        Returns:
        --------------
            final_df (:class:`DataFrame) : new vectorize dataframe  with plot of k
            """
  vecAssembler = VectorAssembler(inputCols=input_columns, outputCol="features")
  final_df = vecAssembler.transform(dataframe)
  ll = np.zeros(kmax)
  for k in np.arange(2,kmax).tolist():
      gm = GaussianMixture(k=k, tol=0.0001,maxIter=maxIter, seed=seed)
      model = gm.fit(final_df)
      summary = model.summary
      ll[k] = summary.logLikelihood
  fig, ax = plt.subplots(1,1, figsize =(8,6))
  ax.plot(np.arange(2,kmax).tolist(),ll[2:kmax])
  ax.set_xlabel('k')
  ax.set_ylabel('ll')
  return final_df

In [ ]:

def transformed(dataframe,k_opt:int,input_columns:list,kmax:int,maxIter:int,seed:int)->tuple:
  """ return a new transformed dataframe  with additional Gaussian information
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : select dataframe  

            input_columns(list) :input columns  

            k_opt(int) : optimal k   
            
            kmax(int) : maximum of k 

            maxIter(int) : maximum iteration 

            seed(int) : seed of Gaussian Mixture cluss cluster model
            
                       
        Returns:
        --------------
            tr_all (tuple) : new transformed dataframe  with additional Gaussian information and summary of all
            """
  final_df=Gaussianplot(dataframe,input_columns,kmax,maxIter,seed)
  gm = GaussianMixture(k=k_opt, maxIter=maxIter, seed=seed)
  model = gm.fit(final_df)

  summary = model.summary
  all_summery={}
  all_summery["Clusters"]=summary.k
  all_summery["Cluster Sizes"]=summary.clusterSizes
  all_summery["Log Likelihood"]=summary.logLikelihood

  weights = model.weights
  all_summery["Model Weights"]=len(weights)

  all_summery["Means"]= model.gaussiansDF.select("mean").head()

  all_summery["Cov"]=model.gaussiansDF.select("cov").head()

  transformed = model.transform(final_df)
  limited =transformed.filter("prediction == 0")
  aggregates = limited.summary("min", "mean", "max")
  all_summery["Total Cases in this Cluster"]=limited.count()
  tr_all=(transformed,all_summery,aggregates)
  return tr_all

# classification and regression



### classification 



#### MultilayerPerceptronClassifier

In [ ]:
def MultilayerPerceptronClassifiermodel(features,classes,folds,train,test,input_columns):
    """ return a Multilayer Perceptron Classifier accuracy 
             
        Parameters:
        -------------- 
                   
            features(:class:`DataFrame) : features column       
            
            classes(int) : number of column target 

            folds(str) : estimate the skill of the model on new data 

            train(:class:`DataFrame) : train dataframe 
            
            test(:class:`DataFrame) :   test dataframe 
            
                       
        Returns:
        --------------
            result (:class:`DataFrame) : return a Multilayer Perceptron Classifier  accuracy
              
            
            """  




    from pyspark.ml.classification import MultilayerPerceptronClassifier
    from pyspark.ml.evaluation import MulticlassClassificationEvaluator
    classifier=MultilayerPerceptronClassifier()
    def FindMtype(classifier):
        # Intstantiate Model
        M = classifier
        # Learn what it is
        Mtype = type(M).__name__
        
        return Mtype
    
    Mtype = FindMtype(classifier)
    

    def IntanceFitModel(Mtype,classifier,classes,features,train):
              
            # specify layers for the neural network:
            # input layer of size features, two intermediate of features+1 and same size as features
            # and output of size number of classes
            # Note: crossvalidator cannot be used here
            features_count = len(features[0][0])
            layers = [features_count, features_count+1, features_count, classes]
            MPC_classifier = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128, seed=1234)
            fitModel = MPC_classifier.fit(train)
            return fitModel
                      
    fitModel = IntanceFitModel(Mtype,classifier,classes,features,train)
  
    # Set the column names to match the external results dataframe that we will join with later:
    columns = ['Classifier', 'Result']
    
    if Mtype in("LinearSVC","GBTClassifier") and classes != 2:
        Mtype = [Mtype] # make this a list
        score = ["N/A"]
        result = spark.createDataFrame(zip(Mtype,score), schema=columns)
    else:
        predictions = fitModel.transform(test)
        MC_evaluator = MulticlassClassificationEvaluator(metricName="accuracy") # redictionCol="prediction",
        accuracy = (MC_evaluator.evaluate(predictions))*100
        Mtype = [Mtype] # make this a string
        score = [str(accuracy)] #make this a string and convert to a list
        result = spark.createDataFrame(zip(Mtype,score), schema=columns)
        result = result.withColumn('Result',result.Result.substr(0, 5))
        
    return result
    #Also returns the fit model important scores or p values

#### OneVsRest

In [ ]:
def OneVsRestmodel(features,classes,folds,train,test,input_columns):
    """ return a One Vs Rest accuracy 
             
        Parameters:
        -------------- 
                   
            features(:class:`DataFrame) : features column       
            
            classes(int) : number of column target 

            folds(str) : estimate the skill of the model on new data 

            train(:class:`DataFrame) : train dataframe 
            
            test(:class:`DataFrame) :   test dataframe 
            
                       
        Returns:
        --------------
            result (:class:`DataFrame) : return One Vs Rest  accuracy
              
            
            """  
    from pyspark.ml.classification import OneVsRest,LogisticRegression
    from pyspark.ml.evaluation import MulticlassClassificationEvaluator
    from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
    classifier=OneVsRest()
    Mtype="OneVsRest"

    def IntanceFitModel(Mtype,classifier,classes,features,train):
        
        if Mtype == "OneVsRest":
            # instantiate the base classifier.
            lr = LogisticRegression()
            # instantiate the One Vs Rest Classifier.
            OVRclassifier = OneVsRest(classifier=lr)
#             fitModel = OVRclassifier.fit(train)
            # Add parameters of your choice here:
            paramGrid = ParamGridBuilder() \
                .addGrid(lr.regParam, [0.1, 0.01]) \
                .build()
            #Cross Validator requires the following parameters:
            crossval = CrossValidator(estimator=OVRclassifier,
                                      estimatorParamMaps=paramGrid,
                                      evaluator=MulticlassClassificationEvaluator(),
                                      numFolds=2) # 3 is best practice
            # Run cross-validation, and choose the best set of parameters.
            fitModel = crossval.fit(train)
            return fitModel

    
    fitModel = IntanceFitModel(Mtype,classifier,classes,features,train)
    
    # Print feature selection metrics
    if fitModel is not None:
        
            # Get Best Model
            BestModel = fitModel.bestModel
            print(" ")
            print('\033[1m' + Mtype + '\033[0m')
            # Extract list of binary models
            models = BestModel.models
            for model in models:
                print('\033[1m' + 'Intercept: '+ '\033[0m',model.intercept,'\033[1m' + '\nCoefficients:'+ '\033[0m',model.coefficients)


    columns = ['Classifier', 'Result']
    
    if Mtype in("LinearSVC","GBTClassifier") and classes != 2:
        Mtype = [Mtype] # make this a list
        score = ["N/A"]
        result = spark.createDataFrame(zip(Mtype,score), schema=columns)
    else:
        predictions = fitModel.transform(test)
        MC_evaluator = MulticlassClassificationEvaluator(metricName="accuracy") # redictionCol="prediction",
        accuracy = (MC_evaluator.evaluate(predictions))*100
        Mtype = [Mtype] # make this a string
        score = [str(accuracy)] #make this a string and convert to a list
        result = spark.createDataFrame(zip(Mtype,score), schema=columns)
        result = result.withColumn('Result',result.Result.substr(0, 5))
        
    return result
    #Also returns the fit model important scores or p values

#### LogisticRegression

In [ ]:
def LogisticRegressionmodel(features,classes,folds,train,test,input_columns) :
    """ return Logistic Regression accuracy 
             
        Parameters:
        -------------- 
                   
            features(:class:`DataFrame) : features column       
            
            classes(int) : number of column target 

            folds(str) : estimate the skill of the model on new data 

            train(:class:`DataFrame) : train dataframe 
            
            test(:class:`DataFrame) :   test dataframe 
            
                       
        Returns:
        --------------
            result (:class:`DataFrame) : return Logistic Regression  accuracy
              
            
            """  
    from pyspark.ml.classification import LogisticRegression
    from pyspark.ml.evaluation import MulticlassClassificationEvaluator
    from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
    classifier=LogisticRegression()
    def FindMtype(classifier):
    
        M = classifier
        Mtype = type(M).__name__
        
        return Mtype
    
    Mtype = FindMtype(classifier)
    

    def IntanceFitModel(Mtype,classifier,classes,features,folds,train):
        


       
  
            # Add parameters of your choice here:
            if Mtype in("LogisticRegression"):
                paramGrid = (ParamGridBuilder() \
#                              .addGrid(classifier.regParam, [0.1, 0.01]) \
                             .addGrid(classifier.maxIter, [10, 15,20])
                             .build())
                

            
            #Cross Validator requires all of the following parameters:
            crossval = CrossValidator(estimator=classifier,
                                      estimatorParamMaps=paramGrid,
                                      evaluator=MulticlassClassificationEvaluator(),
                                      numFolds=folds) # 3 + is best practice
            # Fit Model: Run cross-validation, and choose the best set of parameters.
            fitModel = crossval.fit(train)
            return fitModel
    
    fitModel = IntanceFitModel(Mtype,classifier,classes,features,folds,train)
    
    # Print feature selection metrics
    if fitModel is not None:
        
       
        # Print the coefficients
        if Mtype in("LogisticRegression"):
            # Get Best Model
            BestModel = fitModel.bestModel
            print(" ")
            print('\033[1m' + Mtype + '\033[0m')
            print("Intercept: " + str(BestModel.interceptVector))
            print('\033[1m' + " Top 20 Coefficients"+ '\033[0m')
            print("You should compares these relative to eachother")
            # Convert from numpy array to list
            coeff_array = BestModel.coefficientMatrix.toArray()
            coeff_scores = []
            for x in coeff_array[0]:
                coeff_scores.append(float(x))
            # Then zip with input_columns list and create a df
            result = spark.createDataFrame(zip(input_columns,coeff_scores), schema=['feature','coeff'])
            print(result.orderBy(result["coeff"].desc()).show(truncate=False))
            # Save the coefficient values and the models
            global LR_coefficients
            LR_coefficients = BestModel.coefficientMatrix.toArray()
            global LR_BestModel
            LR_BestModel = BestModel

        
   
    # Set the column names to match the external results dataframe that we will join with later:
    columns = ['Classifier', 'Result']
    
    if Mtype in("LinearSVC","GBTClassifier") and classes != 2:
        Mtype = [Mtype] # make this a list
        score = ["N/A"]
        result = spark.createDataFrame(zip(Mtype,score), schema=columns)
    else:
        predictions = fitModel.transform(test)
        MC_evaluator = MulticlassClassificationEvaluator(metricName="accuracy") # redictionCol="prediction",
        accuracy = (MC_evaluator.evaluate(predictions))*100
        Mtype = [Mtype] # make this a string
        score = [str(accuracy)] #make this a string and convert to a list
        result = spark.createDataFrame(zip(Mtype,score), schema=columns)
        result = result.withColumn('Result',result.Result.substr(0, 5))
        
    return result
    #Also returns the fit model important scores or p values

#### Naive Bayes model

In [ ]:
def NaiveBayesmodel(features,classes,folds,train,test,input_columns) :
    """ return Naive Bayes accuracy 
             
        Parameters:
        -------------- 
                   
            features(:class:`DataFrame) : features column       
            
            classes(int) : number of column target 

            folds(str) : estimate the skill of the model on new data 

            train(:class:`DataFrame) : train dataframe 
            
            test(:class:`DataFrame) :   test dataframe 
            
                       
        Returns:
        --------------
            result (:class:`DataFrame) : return Naive Bayes  accuracy
              
            """    
    from pyspark.ml.classification import NaiveBayes
    from pyspark.ml.evaluation import MulticlassClassificationEvaluator
    from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
    classifier=NaiveBayes()
    
    def FindMtype(classifier):
        # Intstantiate Model
        M = classifier
        # Learn what it is
        Mtype = type(M).__name__
        
        return Mtype
    
    Mtype = FindMtype(classifier)
    

    def IntanceFitModel(Mtype,classifier,classes,features,folds,train):
        
        
      
            if Mtype in("NaiveBayes"):
                paramGrid = (ParamGridBuilder() \
                             .addGrid(classifier.smoothing, [0.0, 0.2, 0.4, 0.6]) \
                             .build())
                
            
            
          
            #Cross Validator requires all of the following parameters:
            crossval = CrossValidator(estimator=classifier,
                                      estimatorParamMaps=paramGrid,
                                      evaluator=MulticlassClassificationEvaluator(),
                                      numFolds=folds) # 3 + is best practice
            # Fit Model: Run cross-validation, and choose the best set of parameters.
            fitModel = crossval.fit(train)
            return fitModel
    
    fitModel = IntanceFitModel(Mtype,classifier,classes,features,folds,train)
    
    # Print feature selection metrics
    if fitModel is not None:
        
       



        # Print the coefficients
       
        if Mtype in("LinearSVC"):
            # Get Best Model
            BestModel = fitModel.bestModel
            print(" ")
            print('\033[1m' + Mtype + '\033[0m')
            print("Intercept: " + str(BestModel.intercept))
            print('\033[1m' + "Top 20 Coefficients"+ '\033[0m')
            print("You should compares these relative to eachother")
#             print("Coefficients: \n" + str(BestModel.coefficients))
            coeff_array = BestModel.coefficients.toArray()
            coeff_scores = []
            for x in coeff_array:
                coeff_scores.append(float(x))
            # Then zip with input_columns list and create a df
            result = spark.createDataFrame(zip(input_columns,coeff_scores), schema=['feature','coeff'])
            print(result.orderBy(result["coeff"].desc()).show(truncate=False))
            # Save the coefficient values and the models
            global LSVC_coefficients
            LSVC_coefficients = BestModel.coefficients.toArray()
            global LSVC_BestModel
            LSVC_BestModel = BestModel
        
   
    # Set the column names to match the external results dataframe that we will join with later:
    columns = ['Classifier', 'Result']
    
    if Mtype in("LinearSVC","GBTClassifier") and classes != 2:
        Mtype = [Mtype] # make this a list
        score = ["N/A"]
        result = spark.createDataFrame(zip(Mtype,score), schema=columns)
    else:
        predictions = fitModel.transform(test)
        MC_evaluator = MulticlassClassificationEvaluator(metricName="accuracy") # redictionCol="prediction",
        accuracy = (MC_evaluator.evaluate(predictions))*100
        Mtype = [Mtype] # make this a string
        score = [str(accuracy)] #make this a string and convert to a list
        result = spark.createDataFrame(zip(Mtype,score), schema=columns)
        result = result.withColumn('Result',result.Result.substr(0, 5))
        
    return result
    #Also returns the fit model important scores or p values

#### Random Forest Classifier model

In [ ]:
def RandomForestClassifiermodel(features,classes,folds,train,test,input_columns) :
    """ return Random Forest Classifier accuracy 
             
        Parameters:
        -------------- 
                   
            features(:class:`DataFrame) : features column       
            
            classes(int) : number of column target 

            folds(str) : estimate the skill of the model on new data 

            train(:class:`DataFrame) : train dataframe 
            
            test(:class:`DataFrame) :   test dataframe 
            
                       
        Returns:
        --------------
            result (:class:`DataFrame) : return Random Forest Classifier  accuracy
              
            """
    from pyspark.ml.classification import RandomForestClassifier
    from pyspark.ml.evaluation import MulticlassClassificationEvaluator
    from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
    classifier=RandomForestClassifier()
    def FindMtype(classifier):
        # Intstantiate Model
        M = classifier
        # Learn what it is
        Mtype = type(M).__name__
        
        return Mtype
    
    Mtype = FindMtype(classifier)
    

    def IntanceFitModel(Mtype,classifier,classes,features,folds,train):
        
       
        
  

            # Add parameters of your choice here:
            if Mtype in("RandomForestClassifier"):
                paramGrid = (ParamGridBuilder() \
                               .addGrid(classifier.maxDepth, [2, 5, 10])
#                                .addGrid(classifier.maxBins, [5, 10, 20])
#                                .addGrid(classifier.numTrees, [5, 20, 50])
                             .build())
                


            

            #Cross Validator requires all of the following parameters:
            crossval = CrossValidator(estimator=classifier,
                                      estimatorParamMaps=paramGrid,
                                      evaluator=MulticlassClassificationEvaluator(),
                                      numFolds=folds) # 3 + is best practice
            # Fit Model: Run cross-validation, and choose the best set of parameters.
            fitModel = crossval.fit(train)
            return fitModel
    
    fitModel = IntanceFitModel(Mtype,classifier,classes,features,folds,train)
    
    # Print feature selection metrics
    if fitModel is not None:




            BestModel = fitModel.bestModel
            print(" ")
            print('\033[1m' + Mtype," Top 20 Feature Importances"+ '\033[0m')
            print("(Scores add up to 1)")
            print("Lowest score is the least important")
            print(" ")
            featureImportances = BestModel.featureImportances.toArray()
            # Convert from numpy array to list
            imp_scores = []
            for x in featureImportances:
                imp_scores.append(float(x))
            # Then zip with input_columns list and create a df
            result = spark.createDataFrame(zip(input_columns,imp_scores), schema=['feature','score'])
            print(result.orderBy(result["score"].desc()).show(truncate=False))
            

            if Mtype in("RandomForestClassifier"):
                global RF_featureimportances
                RF_featureimportances = BestModel.featureImportances.toArray()
                global RF_BestModel
                RF_BestModel = BestModel

  
        
   
    # Set the column names to match the external results dataframe that we will join with later:
    columns = ['Classifier', 'Result']
    
    if Mtype in("LinearSVC","GBTClassifier") and classes != 2:
        Mtype = [Mtype] # make this a list
        score = ["N/A"]
        result = spark.createDataFrame(zip(Mtype,score), schema=columns)
    else:
        predictions = fitModel.transform(test)
        MC_evaluator = MulticlassClassificationEvaluator(metricName="accuracy") # redictionCol="prediction",
        accuracy = (MC_evaluator.evaluate(predictions))*100
        Mtype = [Mtype] # make this a string
        score = [str(accuracy)] #make this a string and convert to a list
        result = spark.createDataFrame(zip(Mtype,score), schema=columns)
        result = result.withColumn('Result',result.Result.substr(0, 5))
        
    return result
    #Also returns the fit model important scores or p values

#### Decision Tree Classifier model

In [ ]:
def DecisionTreeClassifiermodel(features,classes,folds,train,test,input_columns) :
    """ return  Decision Tree Classifier accuracy 
             
        Parameters:
        -------------- 
                   
            features(:class:`DataFrame) : features column       
            
            classes(int) : number of column target 

            folds(str) : estimate the skill of the model on new data 

            train(:class:`DataFrame) : train dataframe 
            
            test(:class:`DataFrame) :   test dataframe 
            
                       
        Returns:
        --------------
            result (:class:`DataFrame) : return Decision Tree Classifier  accuracy
              
            """
    from pyspark.ml.classification import DecisionTreeClassifier
    from pyspark.ml.evaluation import MulticlassClassificationEvaluator
    from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
    classifier=DecisionTreeClassifier()
    def FindMtype(classifier):
        # Intstantiate Model
        M = classifier
        # Learn what it is
        Mtype = type(M).__name__
        
        return Mtype
    
    Mtype = FindMtype(classifier)
    

    def IntanceFitModel(Mtype,classifier,classes,features,folds,train):
        
       
        if Mtype in("LogisticRegression","NaiveBayes","RandomForestClassifier","GBTClassifier","LinearSVC","DecisionTreeClassifier"):
  

            
            # Add parameters of your choice here:
            if Mtype in("DecisionTreeClassifier"):
                paramGrid = (ParamGridBuilder() \
#                              .addGrid(classifier.maxDepth, [2, 5, 10, 20, 30]) \
                             .addGrid(classifier.maxBins, [10, 20, 40, 80, 100]) \
                             .build())
            
            #Cross Validator requires all of the following parameters:
            crossval = CrossValidator(estimator=classifier,
                                      estimatorParamMaps=paramGrid,
                                      evaluator=MulticlassClassificationEvaluator(),
                                      numFolds=folds) # 3 + is best practice
            # Fit Model: Run cross-validation, and choose the best set of parameters.
            fitModel = crossval.fit(train)
            return fitModel
    
    fitModel = IntanceFitModel(Mtype,classifier,classes,features,folds,train)
    
    # Print feature selection metrics
    if fitModel is not None:


            # FEATURE IMPORTANCES
            # Estimate of the importance of each feature.
            # Each feature’s importance is the average of its importance across all trees 
            # in the ensemble The importance vector is normalized to sum to 1. 
            # Get Best Model
            BestModel = fitModel.bestModel
            print(" ")
            print('\033[1m' + Mtype," Top 20 Feature Importances"+ '\033[0m')
            print("(Scores add up to 1)")
            print("Lowest score is the least important")
            print(" ")
            featureImportances = BestModel.featureImportances.toArray()
            # Convert from numpy array to list
            imp_scores = []
            for x in featureImportances:
                imp_scores.append(float(x))
            # Then zip with input_columns list and create a df
            result = spark.createDataFrame(zip(input_columns,imp_scores), schema=['feature','score'])
            print(result.orderBy(result["score"].desc()).show(truncate=False))
            
            # Save the feature importance values and the models
            if Mtype in("DecisionTreeClassifier"):
                global DT_featureimportances
                DT_featureimportances = BestModel.featureImportances.toArray()
                global DT_BestModel
                DT_BestModel = BestModel


        # Print the coefficients

        # Print the Coefficients

        
   
    # Set the column names to match the external results dataframe that we will join with later:
    columns = ['Classifier', 'Result']
    
    if Mtype in("LinearSVC","GBTClassifier") and classes != 2:
        Mtype = [Mtype] # make this a list
        score = ["N/A"]
        result = spark.createDataFrame(zip(Mtype,score), schema=columns)
    else:
        predictions = fitModel.transform(test)
        MC_evaluator = MulticlassClassificationEvaluator(metricName="accuracy") # redictionCol="prediction",
        accuracy = (MC_evaluator.evaluate(predictions))*100
        Mtype = [Mtype] # make this a string
        score = [str(accuracy)] #make this a string and convert to a list
        result = spark.createDataFrame(zip(Mtype,score), schema=columns)
        result = result.withColumn('Result',result.Result.substr(0, 5))
        
    return result
        
    return result
    #Also returns the fit model important scores or p values

#### GBTClassifier

In [ ]:
def GBTClassifiermodel(features,classes,folds,train,test,input_columns):
    """ return GBT Classifier accuracy 
             
        Parameters:
        -------------- 
                   
            features(:class:`DataFrame) : features column       
            
            classes(int) : number of column target 

            folds(str) : estimate the skill of the model on new data 

            train(:class:`DataFrame) : train dataframe 
            
            test(:class:`DataFrame) :   test dataframe 
            
                       
        Returns:
        --------------
            result (:class:`DataFrame) : return GBT Classifier  accuracy
              
            """
    from pyspark.ml.classification import GBTClassifier
    from pyspark.ml.evaluation import MulticlassClassificationEvaluator
    from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
    classifier=GBTClassifier()
    def FindMtype(classifier):
        # Intstantiate Model
        M = classifier
        # Learn what it is
        Mtype = type(M).__name__
        
        return Mtype
    
    Mtype = FindMtype(classifier)
    

    def IntanceFitModel(Mtype,classifier,classes,features,folds,train):
        
        
  

        if Mtype in("LinearSVC","GBTClassifier") and classes != 2: # These classifiers currently only accept binary classification
            print(Mtype," could not be used because PySpark currently only accepts binary classification data for this algorithm")
            return
        if Mtype in("LogisticRegression","NaiveBayes","RandomForestClassifier","GBTClassifier","LinearSVC","DecisionTreeClassifier"):
  

            if Mtype in("GBTClassifier"):
                paramGrid = (ParamGridBuilder() \
#                              .addGrid(classifier.maxDepth, [2, 5, 10, 20, 30]) \
#                              .addGrid(classifier.maxBins, [10, 20, 40, 80, 100]) \
                             .addGrid(classifier.maxIter, [10, 15,50,100])
                             .build())
                


            
            #Cross Validator requires all of the following parameters:
            crossval = CrossValidator(estimator=classifier,
                                      estimatorParamMaps=paramGrid,
                                      evaluator=MulticlassClassificationEvaluator(),
                                      numFolds=folds) # 3 + is best practice
            # Fit Model: Run cross-validation, and choose the best set of parameters.
            fitModel = crossval.fit(train)
            return fitModel
    
    fitModel = IntanceFitModel(Mtype,classifier,classes,features,folds,train)
    
    # Print feature selection metrics
    if fitModel is not None:
        






            # FEATURE IMPORTANCES
            # Estimate of the importance of each feature.
            # Each feature’s importance is the average of its importance across all trees 
            # in the ensemble The importance vector is normalized to sum to 1. 
            # Get Best Model
            BestModel = fitModel.bestModel
            print(" ")
            print('\033[1m' + Mtype," Top 20 Feature Importances"+ '\033[0m')
            print("(Scores add up to 1)")
            print("Lowest score is the least important")
            print(" ")
            featureImportances = BestModel.featureImportances.toArray()
            # Convert from numpy array to list
            imp_scores = []
            for x in featureImportances:
                imp_scores.append(float(x))
            # Then zip with input_columns list and create a df
            result = spark.createDataFrame(zip(input_columns,imp_scores), schema=['feature','score'])
            print(result.orderBy(result["score"].desc()).show(truncate=False))
            
            # Save the feature importance values and the models

            if Mtype in("GBTClassifier"):
                global GBT_featureimportances
                GBT_featureimportances = BestModel.featureImportances.toArray()
                global GBT_BestModel
                GBT_BestModel = BestModel


        
        
   
    # Set the column names to match the external results dataframe that we will join with later:
    columns = ['Classifier', 'Result']
    
    if Mtype in("LinearSVC","GBTClassifier") and classes != 2:
        Mtype = [Mtype] # make this a list
        score = ["N/A"]
        result = spark.createDataFrame(zip(Mtype,score), schema=columns)
    else:
        predictions = fitModel.transform(test)
        MC_evaluator = MulticlassClassificationEvaluator(metricName="accuracy") # redictionCol="prediction",
        accuracy = (MC_evaluator.evaluate(predictions))*100
        Mtype = [Mtype] # make this a string
        score = [str(accuracy)] #make this a string and convert to a list
        result = spark.createDataFrame(zip(Mtype,score), schema=columns)
        result = result.withColumn('Result',result.Result.substr(0, 5))
        
    return result
    #Also returns the fit model important scores or p values

#### Linear SVC model

In [ ]:
def LinearSVCmodel(features,classes,folds,train,test,input_columns):
    """ return Linear SVC accuracy 
             
        Parameters:
        -------------- 
                   
            features(:class:`DataFrame) : features column       
            
            classes(int) : number of column target 

            folds(int) : estimate the skill of the model on new data 

            train(:class:`DataFrame) : train dataframe 
            
            test(:class:`DataFrame) :   test dataframe 
            
                       
        Returns:
        --------------
            result (:class:`DataFrame) : return Linear SVC  accuracy
              
            """
    from pyspark.ml.classification import LinearSVC
    from pyspark.ml.evaluation import MulticlassClassificationEvaluator
    from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
    classifier=LinearSVC()
    def FindMtype(classifier):
        # Intstantiate Model
        M = classifier
        # Learn what it is
        Mtype = type(M).__name__
        
        return Mtype
    
    Mtype = FindMtype(classifier)
    

    def IntanceFitModel(Mtype,classifier,classes,features,folds,train):
        


        if Mtype in("LinearSVC") and classes != 2: # These classifiers currently only accept binary classification
            print(Mtype," could not be used because PySpark currently only accepts binary classification data for this algorithm")
            return
        if Mtype in("LogisticRegression","NaiveBayes","RandomForestClassifier","GBTClassifier","LinearSVC","DecisionTreeClassifier"):
  


            if Mtype in("LinearSVC"):
                paramGrid = (ParamGridBuilder() \
                             .addGrid(classifier.maxIter, [10, 15]) \
                             .addGrid(classifier.regParam, [0.1, 0.01]) \
                             .build())

            
            #Cross Validator requires all of the following parameters:
            crossval = CrossValidator(estimator=classifier,
                                      estimatorParamMaps=paramGrid,
                                      evaluator=MulticlassClassificationEvaluator(),
                                      numFolds=folds) # 3 + is best practice
            # Fit Model: Run cross-validation, and choose the best set of parameters.
            fitModel = crossval.fit(train)
            return fitModel
    
    fitModel = IntanceFitModel(Mtype,classifier,classes,features,folds,train)
    
    # Print feature selection metrics
    if fitModel is not None:
        

        if Mtype in("LinearSVC"):
            # Get Best Model
            BestModel = fitModel.bestModel
            print(" ")
            print('\033[1m' + Mtype + '\033[0m')
            print("Intercept: " + str(BestModel.intercept))
            print('\033[1m' + "Top 20 Coefficients"+ '\033[0m')
            print("You should compares these relative to eachother")
#             print("Coefficients: \n" + str(BestModel.coefficients))
            coeff_array = BestModel.coefficients.toArray()
            coeff_scores = []
            for x in coeff_array:
                coeff_scores.append(float(x))
            # Then zip with input_columns list and create a df
            result = spark.createDataFrame(zip(input_columns,coeff_scores), schema=['feature','coeff'])
            print(result.orderBy(result["coeff"].desc()).show(truncate=False))
            # Save the coefficient values and the models
            global LSVC_coefficients
            LSVC_coefficients = BestModel.coefficients.toArray()
            global LSVC_BestModel
            LSVC_BestModel = BestModel
        
   
    # Set the column names to match the external results dataframe that we will join with later:
    columns = ['Classifier', 'Result']
    
    if Mtype in("LinearSVC","GBTClassifier") and classes != 2:
        Mtype = [Mtype] # make this a list
        score = ["N/A"]
        result = spark.createDataFrame(zip(Mtype,score), schema=columns)
    else:
        predictions = fitModel.transform(test)
        MC_evaluator = MulticlassClassificationEvaluator(metricName="accuracy") # redictionCol="prediction",
        accuracy = (MC_evaluator.evaluate(predictions))*100
        Mtype = [Mtype] # make this a string
        score = [str(accuracy)] #make this a string and convert to a list
        result = spark.createDataFrame(zip(Mtype,score), schema=columns)
        result = result.withColumn('Result',result.Result.substr(0, 5))
        
    return result
    #Also returns the fit model important scores or p values

## test with  or without treat outliers,treat negative values

In [ ]:
def modeltest(dataframe,classifier,dependent_var:str,input_columns:list,trainsplite:int,testsplite:int,seed:int,folds:int,treat_outliers:bool,treat_neg_values:bool):
    """ return all model accuracy 
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : dataframe        
            
            classifier : name of model incude () 

            dependent_var(str) : estimate the skill of the model on new data 

            input_columns(list) : input_columns
            
            trainsplite(int) :   number train splite

            testsplite(int) :   number test splite

            seed(int) : seed of train and test
            
            folds(int) : estimate the skill of the model on new data
            
            treat_outliers(bool) : fix outliers 

            treat_neg_values(bool) :fix negative value 
                       
        Returns:
        --------------
            result (:class:`DataFrame) : return all  accuracy
              
            """
    from pyspark.sql.functions import countDistinct
    def FindMtype(classifier):
        # Intstantiate Model
        M = classifier
        # Learn what it is
        Mtype = type(M).__name__
        return Mtype
    Mtype = FindMtype(classifier)
  

    final_data=MLClassifierDFPrep(dataframe,input_columns,dependent_var,treat_outliers=treat_outliers,treat_neg_values=treat_neg_values)
    class_count = dataframe.select(countDistinct(dependent_var)).collect()
    classes = class_count[0][0]
    classes

    train,test = final_data.randomSplit([trainsplite,testsplite],seed)
    features = final_data.select(['features']).collect()
    
    #set up your results table
    columns = ['Classifier', 'Result']
    vals = [("Place Holder","N/A")]
    results = spark.createDataFrame(vals, columns)

    if Mtype=="GBTClassifier":
        new_result = GBTClassifiermodel(features,classes,folds,train,test)
    elif Mtype=="RandomForestClassifier":
        new_result = RandomForestClassifiermodel(features,classes,folds,train,test)
    elif Mtype=="OneVsRest":
        new_result = OneVsRestmodel(features,classes,folds,train,test)
    elif Mtype=="LogisticRegression":
        new_result = LogisticRegressionmodel(features,classes,folds,train,test)
    elif Mtype=="NaiveBayes":
        new_result = NaiveBayesmodel(features,classes,folds,train,test)
    elif Mtype=="DecisionTreeClassifier":
        new_result = DecisionTreeClassifiermodel(features,classes,folds,train,test)
    elif Mtype=="LinearSVC":
        new_result = LinearSVCmodel(features,classes,folds,train,test)
    elif Mtype=="MultilayerPerceptronClassifier":
      new_result = MultilayerPerceptronClassifiermodel(features,classes,folds,train,test) 
    
    results = results.union(new_result)
    results = results.where("Classifier!='Place Holder'")
    return results

In [ ]:
def  FeatureSelectionrandommodel(dataframe,classifier,input_columns:list,dependent_var:str,trainsplite:int,testsplite:int,seed:int,folds:int,start:int,step:int):  
    """ return all model accuracy  with feature selection
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : dataframe        
            
            classifier : name of model incude () 

            dependent_var(str) : estimate the skill of the model on new data 

            input_columns(list) : input_columns
            
            trainsplite(int) :   number train splite

            testsplite(int) :   number test splite

            seed(int) : seed of train and test
            
            folds(int) : estimate the skill of the model on new data
            
            treat_outliers(bool) : fix outliers 

            treat_neg_values(bool) :fix negative value 
                       
        Returns:
        --------------
            result (:class:`DataFrame) : return all  accuracy
              
            """
    from pyspark.ml.feature import VectorSlicer
    from pyspark.ml.feature import ChiSqSelector
    from pyspark.sql.functions import countDistinct
    from pyspark.ml.linalg import Vectors
    import numpy as np
    def FindMtype(classifier):
          # Intstantiate Model
          M = classifier
          # Learn what it is
          Mtype = type(M).__name__

          return Mtype

    Mtype = FindMtype(classifier)
    test2_data = MLClassifierDFPrep(dataframe,input_columns,dependent_var,treat_outliers=True,treat_neg_values=True)  
    class_count = dataframe.select(countDistinct(dependent_var)).collect()
    classes = class_count[0][0]
    #Select the top n features and view results
    maximum = len(input_columns)
    for n in np.arange(start, maximum+1 , step).tolist():
        print("Testing top n = ",n," features")

        if Mtype in("DecisionTreeClassifier", "GBTClassifier","RandomForestClassifier"):
            best_n_features = RF_featureimportances.argsort()[-n:][::-1]
            best_n_features= best_n_features.tolist() # convert to a list
            vs = VectorSlicer(inputCol="features", outputCol="best_features", indices=best_n_features)
            bestFeaturesDf = vs.transform(test2_data)

        else:
            selector = ChiSqSelector(numTopFeatures=n, featuresCol="features",
                                outputCol="selectedFeatures", labelCol="label")
            bestFeaturesDf = selector.fit(test2_data).transform(test2_data)
            bestFeaturesDf = bestFeaturesDf.select("label","selectedFeatures")
            bestFeaturesDf = bestFeaturesDf.withColumnRenamed("selectedFeatures","features")

        # Collect features
        features = bestFeaturesDf.select(['features']).collect()

        # Split
        train,test = bestFeaturesDf.randomSplit([trainsplite,testsplite],seed)

        # Specify folds


        #set up your results table
        columns = ['Classifier', 'Result']
        vals = [("Place Holder","N/A")]
        results = spark.createDataFrame(vals, columns)


        if Mtype=="GBTClassifier":
            new_result = GBTClassifiermodel(features,classes,folds,train,test)
        elif Mtype=="RandomForestClassifier":
            new_result = RandomForestClassifiermodel(features,classes,folds,train,test)
        elif Mtype=="OneVsRest":
            new_result = OneVsRestmodel(features,classes,folds,train,test)
        elif Mtype=="LogisticRegression":
            new_result = LogisticRegressionmodel(features,classes,folds,train,test)
        elif Mtype=="NaiveBayes":
            new_result = NaiveBayesmodel(features,classes,folds,train,test)
        elif Mtype=="DecisionTreeClassifier":
            new_result = DecisionTreeClassifiermodel(features,classes,folds,train,test)
        elif Mtype=="LinearSVC":
            new_result = LinearSVCmodel(features,classes,folds,train,test)
        elif Mtype=="MultilayerPerceptronClassifier":
          new_result = MultilayerPerceptronClassifiermodel(features,classes,folds,train,test) 

        results = results.union(new_result)
        results = results.where("Classifier!='Place Holder'")
        return results

##text minning

### return dataframe pipeline with  regex_tokenizer,remover,indexer satges.


In [ ]:
def data_prep_pl(dataframe,inputCols:list,inputColtarget:str):
  """ return dataframe pipeline with  regex_tokenizer,remover,indexer satges.
             
      Parameters:
      -------------- 
                   
      dataframe(:class:`DataFrame) : select dataframe                       
      
      inputCols(list):  input columns of datafram
    
      inputColtarget(str):  ccolumn target of dataframe
       

  

            
      Returns:
      -------------
      feature_data (:class:`DataFrame) : new dataframe with trim a column


  
            
  """ 
  from pyspark.ml.feature import CountVectorizer,StringIndexer, RegexTokenizer,StopWordsRemover
  from pyspark.sql.functions import col, udf,regexp_replace,isnull
  from pyspark.sql.types import StringType,IntegerType
  from pyspark.ml import Pipeline 
  feature_data=dataframe
  indexer = StringIndexer(inputCol=inputColtarget, outputCol="label")
  
  for count, inputCol in enumerate(inputCols):
        inputCol=inputCol
        outputColRE="words"+str(count+1)
        filteredcolumn="filtered"+str(count+1)
        # Tokenize
        regex_tokenizer = RegexTokenizer(inputCol=inputCol, outputCol=outputColRE, pattern="\\W")

        remover = StopWordsRemover(inputCol=regex_tokenizer.getOutputCol(), outputCol=filteredcolumn)

              # Zero Index Label Column
        
  
        # Create the Pipeline
        pipeline = Pipeline(stages=[regex_tokenizer,remover])
        data_prep_pl = pipeline.fit(feature_data)
              

        feature_data = data_prep_pl.transform(feature_data)
  pipeline = Pipeline(stages=[indexer])
  data_prep_pl = pipeline.fit(feature_data)
              

  feature_data = data_prep_pl.transform(feature_data)
  return feature_data

In [ ]:
def hashingTFIDF(feature_data,inputCols:list,list,numFeatures:int)->tuple: 
  """ return HTFfeaturizedData,TFIDFfeaturizedData
             
        Parameters:
        -------------- 
                   
            feature_data(:class:`DataFrame) : select data_prep_pl           
            
                       
            inputCols(list):  filtered columns  for next uses.


            numFeatures(str): number of feature 

         

  

            
        Returns:
        --------------
            (tuple) : HTFfeaturizedData,TFIDFfeaturizedData



            
  """ 
  HTFfeaturizedData=feature_data
  for count, inputCol in enumerate(inputCols):
          inputCol="filtered"+str(count+1)
          outputCol="rawfeatures"+str(count+1)
 
          hashingTF = HashingTF(inputCol=inputCol, outputCol=outputCol, numFeatures=numFeatures)
          HTFfeaturizedData = hashingTF.transform(HTFfeaturizedData)
          HTFfeaturizedData.name = 'HTFfeaturizedData' #We will use later for printing
  TFIDFfeaturizedData=HTFfeaturizedData
  for count, inputCol in enumerate(inputCols):
      inputCol="rawfeatures"+str(count+1)
      outputCol="features"+str(count+1)
      idf = IDF(inputCol=hashingoutputCol, outputCol=outputCol)
      idfModel = idf.fit(HTFfeaturizedData)
      TFIDFfeaturizedData = idfModel.transform(TFIDFfeaturizedData)
      TFIDFfeaturizedData.name = 'TFIDFfeaturizedData'
  for count, inputCols in enumerate(inputCols): 
      inputCol="rawfeatures"+str(count+1)
      outputCol="features"+str(count+1)  
      HTFfeaturizedData = HTFfeaturizedData.withColumnRenamed(inputCol,outputCol)
   
  return HTFfeaturizedData,TFIDFfeaturizedData

In [ ]:
def words2vec(feature_data,inputCol:list,inputColtarget:str): 
    """ return W2VfeaturizedData
             
        Parameters:
        -------------- 
                   
            feature_data(:class:`DataFrame) : select dataframe from data_prep_pl                      
                       
            inputCol(list):  input coloumns.

            inputColtarget(str):  ccolumn target of dataframe


  

            
        Returns:
        --------------
            W2VfeaturizedData(:class:`DataFrame) : return word to vector dataframe

      
    """ 
    from pyspark.ml.feature import CountVectorizer,StringIndexer, RegexTokenizer,StopWordsRemover
    from pyspark.sql.functions import col, udf,regexp_replace,isnull
    from pyspark.sql.types import StringType,IntegerType

    W2VfeaturizedData=feature_data
    for count, inputCol in enumerate(inputCols):
          inputCol="filtered"+str(count+1)
          outputCol="features"+str(count+1)
          word2Vec = Word2Vec(vectorSize=3, minCount=0, inputCol=inputCol, outputCol=outputCol)
          model = word2Vec.fit(W2VfeaturizedData)

          W2VfeaturizedData = model.transform(W2VfeaturizedData)
          
    scaled_data=W2VfeaturizedData
    for count, inputCol in enumerate(inputCols):
          inputCol="features"+str(count+1)
          outputCol="rawfeatures"+str(count+1)
          # W2Vec Dataframes typically has negative values so we will correct for that here so that we can use the Naive Bayes classifier
          scaler = MinMaxScaler(inputCol=inputCol, outputCol=outputCol)

          # Compute summary statistics and generate MinMaxScalerModel
          scalerModel = scaler.fit(scaled_data)

          # rescale each feature to range [min, max].
          scaled_data = scalerModel.transform(scaled_data)
    scaled_data = scaled_data.drop("features1","features2")
    for count, inputCol in enumerate(inputCols):
        inputCol="rawfeatures"+str(count+1)
        outputCol="features"+str(count+1)    
        scaled_data = scaled_data.withColumnRenamed(inputCol,outputCol)

    W2VfeaturizedData.name = 'W2VfeaturizedData'
    W2VfeaturizedData=scaled_data
    return W2VfeaturizedData


In [ ]:
def words2vec(feature_data,vectorSize:int,minCount:int,filteredcolumns:list,featureCols:list,middleputCols:list,inputColtarget:str): 
    """ return W2VfeaturizedData
             
        Parameters:
        -------------- 
                   
            feature_data(:class:`DataFrame) : select dataframe from data_prep_pl                      
                       
            filteredcolumns(str):  filtered columns  for next uses.

            featureCols(str):  main features for next step 

            middleputCols(str):  middle column for renaming usage

            inputColtarget(str):  ccolumn target of dataframe


  

            
        Returns:
        --------------
            (tuple) : HTFfeaturizedData,TFIDFfeaturizedData

        Examlpe:
        --------------
            middleputCols=["rawfeatures1","rawfeatures2"]
            featureCols=["features1","features2"]

            
    """ 
    
    W2VfeaturizedData=feature_data
    for count, filteredcolumn in enumerate(filteredcolumns):
          inputCol=filteredcolumn
          outputCol=featureCols[count]  
          word2Vec = Word2Vec(vectorSize=vectorSize, minCount=minCount, inputCol=inputCol, outputCol=outputCol)
          model = word2Vec.fit(W2VfeaturizedData)

          W2VfeaturizedData = model.transform(W2VfeaturizedData)
          
    scaled_data=W2VfeaturizedData
    for count, featureCol in enumerate(featureCols):
          inputCol=featureCol
          outputCol=middleputCols[count]
          # W2Vec Dataframes typically has negative values so we will correct for that here so that we can use the Naive Bayes classifier
          scaler = MinMaxScaler(inputCol=inputCol, outputCol=outputCol)

          # Compute summary statistics and generate MinMaxScalerModel
          scalerModel = scaler.fit(scaled_data)

          # rescale each feature to range [min, max].
          scaled_data = scalerModel.transform(scaled_data)
    scaled_data = scaled_data.drop("features1","features2")
    for count, middleputCol in enumerate(middleputCols):
        inputCol=middleputCol
        outputCol=featureCols[count]    
        scaled_data = scaled_data.withColumnRenamed(inputCol,outputCol)

    W2VfeaturizedData.name = 'W2VfeaturizedData'
    W2VfeaturizedData=scaled_data
    return W2VfeaturizedData


In [ ]:
def featureDF(dataframe,classifier,trainsplite:int,testsplite:int,seed:int,folds:int,featureDFs:tuple):
    """ return all model accuracy 
             
        Parameters:
        -------------- 
                   
            dataframe(:class:`DataFrame) : dataframe        
            
            classifier : name of model incude () 

            
            trainsplite(int) :   number train splite

            testsplite(int) :   number test splite

            seed(int) : seed of train and test
            
            folds(int) : estimate the skill of the model on new data
            
            featureDFs(tuple) : HTFfeaturizedData,TFIDFfeaturizedData,W2VfeaturizedData

            
                       
        Returns:
        --------------
            result (:class:`DataFrame) : return all  accuracy
              
    """
    from pyspark.sql.functions import countDistinct
    def FindMtype(classifier):
        # Intstantiate Model
        M = classifier
        # Learn what it is
        Mtype = type(M).__name__
        return Mtype
    Mtype = FindMtype(classifier)
    for featureDF in featureDFs:

      print('\033[1m' + featureDF.name," Results:"+ '\033[0m')
      train, test = featureDF.randomSplit([trainsplite, testsplite],seed = seed)
      features = featureDF.select(['features']).collect()
      # Learn how many classes there are in order to specify evaluation type based on binary or multi and turn the df into an object
      class_count = featureDF.select(countDistinct("label")).collect()
      classes = class_count[0][0]

      #set up your results table
      columns = ['Classifier', 'Result']
      vals = [("Place Holder","N/A")]
      results = spark.createDataFrame(vals, columns)


      if Mtype=="GBTClassifier":
          new_result = GBTClassifiermodel(features,classes,folds,train,test)
      elif Mtype=="RandomForestClassifier":
          new_result = RandomForestClassifiermodel(features,classes,folds,train,test)
      elif Mtype=="OneVsRest":
          new_result = OneVsRestmodel(features,classes,folds,train,test)
      elif Mtype=="LogisticRegression":
          new_result = LogisticRegressionmodel(features,classes,folds,train,test)
      elif Mtype=="NaiveBayes":
          new_result = NaiveBayesmodel(features,classes,folds,train,test)
      elif Mtype=="DecisionTreeClassifier":
          new_result = DecisionTreeClassifiermodel(features,classes,folds,train,test)
      elif Mtype=="LinearSVC":
          new_result = LinearSVCmodel(features,classes,folds,train,test)
      elif Mtype=="MultilayerPerceptronClassifier":
        new_result = MultilayerPerceptronClassifiermodel(features,classes,folds,train,test) 
    

## regression model

### prepration data

In [ ]:
def MLRegressDFPrep(df,input_columns,dependent_var,treat_outliers=True):
    """ return a dataframe with data prepration ( checking skewness and outliers)
             
        Parameters:
        -------------- 
                   
            df(:class:`DataFrame) : select dataframe        
            
            inputCols(str) : name of input columns 

            dependent_var(str) : name of dependent columns 

            treat_outliers(bool) : modify dataframe for outliers

            
            
                       
        Returns:
        --------------
            final_data (:class:`DataFrame) : the  dataframe  with data prepration 
              
            
            """  
  
    renamed = df.withColumnRenamed(dependent_var,'label')
    
    # Make sure dependent variable is numeric and change if it's not
    if str(renamed.schema['label'].dataType) != 'IntegerType':
        renamed = renamed.withColumn("label", renamed["label"].cast(FloatType()))
    
   # Convert all string type data in the input column list to numeric
    # Otherwise the Algorithm will not be able to process it
    numeric_inputs = []
    string_inputs = []
    for column in input_columns:
        if str(renamed.schema[column].dataType) == 'StringType':
            new_col_name = column+"_num"
            string_inputs.append(new_col_name)
        else:
            numeric_inputs.append(column)
            indexed = renamed
            
    if len(string_inputs) != 0: # If the datafraem contains string types
        for column in input_columns:
            if str(renamed.schema[column].dataType) == 'StringType':
                indexer = StringIndexer(inputCol=column, outputCol=column+"_num") 
                indexed = indexer.fit(renamed).transform(renamed)
    else:
        indexed = renamed
        
            
    if treat_outliers == True:
        print("We are correcting for non normality now!")
        # empty dictionary d
        d = {}
        # Create a dictionary of quantiles
        for col in numeric_inputs: 
            d[col] = indexed.approxQuantile(col,[0.01,0.99],0.25) #if you want to make it go faster increase the last number
        #Now fill in the values
        for col in numeric_inputs:
            skew = indexed.agg(skewness(indexed[col])).collect() #check for skewness
            skew = skew[0][0]
            # This function will floor, cap and then log+1 (just in case there are 0 values)
            if skew > 1:
                indexed = indexed.withColumn(col, \
                log(when(df[col] < d[col][0],d[col][0])\
                .when(indexed[col] > d[col][1], d[col][1])\
                .otherwise(indexed[col] ) +1).alias(col))
                print(col+" has been treated for positive (right) skewness. (skew =)",skew,")")
            elif skew < -1:
                indexed = indexed.withColumn(col, \
                exp(when(df[col] < d[col][0],d[col][0])\
                .when(indexed[col] > d[col][1], d[col][1])\
                .otherwise(indexed[col] )).alias(col))
                print(col+" has been treated for negative (left) skewness. (skew =",skew,")")

            
    # Produce a warning if there are negative values in the dataframe that Naive Bayes cannot be used. 
    # Note: we only need to check the numeric input values since anything that is indexed won't have negative values
    minimums = df.select([min(c).alias(c) for c in df.columns if c in numeric_inputs]) # Calculate the mins for all columns in the df
    min_array = minimums.select(array(numeric_inputs).alias("mins")) # Create an array for all mins and select only the input cols
    df_minimum = min_array.select(array_min(min_array.mins)).collect() # Collect golobal min as Python object
    df_minimum = df_minimum[0][0] # Slice to get the number itself

    features_list = numeric_inputs + string_inputs
    assembler = VectorAssembler(inputCols=features_list,outputCol='features')
    output = assembler.transform(indexed).select('features','label')

#     final_data = output.select('features','label') #drop everything else
    
    # Now check for negative values and ask user if they want to correct that? 
    if df_minimum < 0:
        print(" ")
        print("WARNING: The Naive Bayes Classifier will not be able to process your dataframe as it contains negative values")
        print(" ")
    
    if treat_neg_values == True:
        print("You have opted to correct that by rescaling all your features to a range of 0 to 1")
        print(" ")
        print("We are rescaling you dataframe....")
        scaler = MinMaxScaler(inputCol="features", outputCol="scaledFeatures")

        # Compute summary statistics and generate MinMaxScalerModel
        global scalerModel
        scalerModel = scaler.fit(output)

        # rescale each feature to range [min, max].
        scaled_data = scalerModel.transform(output)
        final_data = scaled_data.select('label','scaledFeatures')
        final_data = final_data.withColumnRenamed('scaledFeatures','features')
        print("Done!")

    else:
        print("You have opted not to correct that therefore you will not be able to use to Naive Bayes classifier")
        print("We will return the dataframe unscaled.")
        final_data = output
    
    return final_data

#### Linear Regression model

In [ ]:
def LinearRegressionmodel(dataframe,input_columns:list,dependent_var:str,trainesplite:int,testsplite:int,treat_outliers=True):
    """ return Linear Regression accuracy 
             
        Parameters:
        -------------- 
            dataframe  (:class:`DataFrame):main dataframe

            dependent_var(str) : dependent varvariable column  

            input_columns(list)  : name of input columns 
            
            train(int) : train splite  
            
            test(int) :   test splite 

            treat_outliers(bool) : treating outliers
            
                       
        Returns:
        --------------
            result (:class:`DataFrame) : return Linear Regression accuracy 
              
            """
    final_data=MLRegressDFPrep(dataframe,input_columns,dependent_var,treat_outliers)
    train,test = final_data.randomSplit([trainesplite,testsplite])
    regressor=LinearRegression()
    def FindMtype(regressor):
        # Intstantiate Model
        M = regressor
        # Learn what it is
        Mtype = type(M).__name__
        
        return Mtype
    
    Mtype = FindMtype(regressor)
#     print('\033[1m' + Mtype + ':' + '\033[0m')


    if Mtype == "LinearRegression":
        
           Mtype = FindMtype(regressor)
#     print('\033[1m' + Mtype + ':' + '\033[0m')


    if Mtype == "LinearRegression":
        
        #first without cross val
        fitModel = regressor.fit(train)

        # Load the Summary
        trainingSummary = fitModel.summary
        
        # Print the coefficients and intercept for linear regression
        print('\033[1m' + "Linear Regression Model Summary without cross validation:"+ '\033[0m')
        print(" ")
        print("Coefficients: %s" % str(fitModel.coefficients))
        print("Intercept: %s" % str(fitModel.intercept))
        print("")

        # Summarize the model over the training set and print out some metrics
        print("numIterations: %d" % trainingSummary.totalIterations)
        print("objectiveHistory: (scaled loss + regularization) at each iteration \n %s" % str(trainingSummary.objectiveHistory))
        print("")
        
        # Print the Errors
        print("Training RMSE: %f" % trainingSummary.rootMeanSquaredError)
        print("Training r2: %f" % trainingSummary.r2)
        print("")
        

        # Now load the test results
        test_results = fitModel.evaluate(test)

        # And print them
        print("Test RMSE: {}".format(test_results.rootMeanSquaredError))
        print("Test r2: {}".format(test_results.r2))
        print("")
        
        #Now train with cross val
        paramGrid = (ParamGridBuilder() \
#              .addGrid(regressor.maxIter, [10, 15]) \
             .addGrid(regressor.regParam, [0.1, 0.01]) \
             .build())
        
        #Evaluator
        revaluator = RegressionEvaluator(metricName="rmse")
        
        #Cross Validator requires all of the following parameters:
        crossval = CrossValidator(estimator=regressor,
                                  estimatorParamMaps=paramGrid,
                                  evaluator=revaluator,
                                  numFolds=2) # 3 is best practice
        
        print('\033[1m' + "Linear Regression Model Summary WITH cross validation:"+ '\033[0m')
        print(" ")
        # Run cross validations
        fitModel = crossval.fit(train)
        
        # Get Model Summary Statistics
        ModelSummary = fitModel.bestModel.summary
        print("Coefficient Standard Errors: " + str(ModelSummary.coefficientStandardErrors))
        print(" ")
        print("P Values: " + str(ModelSummary.pValues)) # Last element is the intercept
        print(" ")
        
        global LR_Pvalues
        LR_Pvalues = ModelSummary.pValues
        
        #save model
        global LR_BestModel 
        LR_BestModel = fitModel.bestModel
        
        # Use test set here so we can measure the accuracy of our model on new data
        ModelPredictions = fitModel.transform(test)
        
        # cvModel uses the best model found from the Cross Validation
        # Evaluate best model
        test_results = revaluator.evaluate(ModelPredictions)
        print('RMSE:', test_results)
    
        # Set the column names to match the external results dataframe that we will join with later:
        columns = ['Regressor', 'Result']
        
        # Format results and return
        rmse_str = [str(test_results)] #make this a string and convert to a list
        Mtype = [Mtype] #make this a string
        result = spark.createDataFrame(zip(Mtype,rmse_str), schema=columns)
        result = result.withColumn('Result',result.Result.substr(0, 5))
        return result

###  Random Forest Regressor

In [ ]:
def RandomForestRegressormodel(dataframe,input_columns,dependent_var,trainesplite,testsplite,treat_outliers=True):
    """ return Random Forest Regressor accuracy 
             
        Parameters:
        -------------- 
            dataframe  (:class:`DataFrame):main dataframe

            dependent_var(str) : dependent varvariable column  

            input_columns(list)  : name of input columns 
            
            train(int) : train splite  
            
            test(int) :   test splite 

            treat_outliers(bool) : treating outliers
                       
        Returns:
        --------------
            result (:class:`DataFrame) : return Random Forest Regressor accuracy 
              
            """
    final_data=MLRegressDFPrep(dataframe,input_columns,dependent_var,treat_outliers)
    train,test = final_data.randomSplit([trainesplite,testsplite])
    regressor=RandomForestRegressor()
    def FindMtype(regressor):
        # Intstantiate Model
        M = regressor
        # Learn what it is
        Mtype = type(M).__name__
        
        return Mtype
    
    Mtype = FindMtype(regressor)
#     print('\033[1m' + Mtype + ':' + '\033[0m')




    if not Mtype == "LinearRegression":

        # Add parameters of your choice here:
        if Mtype in("RandomForestRegressor"):
            paramGrid = (ParamGridBuilder() \
#                            .addGrid(regressor.maxDepth, [2, 5, 10])
#                            .addGrid(regressor.maxBins, [5, 10, 20])
                           .addGrid(regressor.numTrees, [5, 20])
                         .build())

        # Add parameters of your choice here:
       

        #Cross Validator requires all of the following parameters:
        crossval = CrossValidator(estimator=regressor,
                                  estimatorParamMaps=paramGrid,
                                  evaluator=RegressionEvaluator(metricName="rmse"),
                                  numFolds=2) # 3 is best practice
        # Fit Model: Run cross-validation, and choose the best set of parameters.
        fitModel = crossval.fit(train)
        
        # Get Best Model
        BestModel = fitModel.bestModel

        # FEATURE IMPORTANCES
        # Estimate of the importance of each feature.
        # Each feature’s importance is the average of its importance across all trees 
        # in the ensemble The importance vector is normalized to sum to 1. 
        print(" ")
        print('\033[1m' + Mtype," Feature Importances"+ '\033[0m')
        print("(Scores add up to 1)")
        print("Lowest score is the least important")
        print(" ")
        featureImportances = BestModel.featureImportances.toArray()
        # Convert from numpy array to list
        imp_scores = []
        for x in featureImportances:
            imp_scores.append(float(x))
        # Then zip with input_columns list and create a df
        result = spark.createDataFrame(zip(input_columns,imp_scores), schema=['feature','score'])
        print(result.orderBy(result["score"].desc()).show(truncate=False))
        
        #Create Global Variables for feature importances and models

        if Mtype in("RandomForestRegressor"):
            global RF_featureImportances
            RF_featureImportances = BestModel.featureImportances.toArray()
            global RF_BestModel 
            RF_BestModel = fitModel.bestModel
                    
        # Set the column names to match the external results dataframe that we will join with later:
        columns = ['Regressor', 'Result']
        
        # Make predictions.
        predictions = fitModel.transform(test)
        # Select (prediction, true label) and compute test error
        evaluator = RegressionEvaluator(metricName="rmse")
        rmse = evaluator.evaluate(predictions)
        rmse_str = [str(rmse)] #make this a string and convert to a list
        Mtype = [Mtype] #make this a string
        result = spark.createDataFrame(zip(Mtype,rmse_str), schema=columns)
        # Clean up the Result column and output
        result = result.withColumn('Result',result.Result.substr(0, 5))
        return result

### GBT Regressor 

In [ ]:
def GBTRegressormodel(dataframe,input_columns,dependent_var,trainesplite,testsplite,treat_outliers):
    """ return GBT Regressor accuracy 
             
        Parameters:
        -------------- 
            dataframe  (:class:`DataFrame):main dataframe

            dependent_var(str) : dependent varvariable column  

            input_columns(list)  : name of input columns 
            
            train(int) : train splite  
            
            test(int) :   test splite 

            treat_outliers(bool) : treating outliers
            
                       
        Returns:
        --------------
            result (:class:`DataFrame) : return GBT Regressor accuracy 
              
            """
    final_data=MLRegressDFPrep(dataframe,input_columns,dependent_var,treat_outliers)
    train,test = final_data.randomSplit([trainesplite,testsplite])
    regressor=GBTRegressor()
    def FindMtype(regressor):
        # Intstantiate Model
        M = regressor
        # Learn what it is
        Mtype = type(M).__name__
        
        return Mtype
    
    Mtype = FindMtype(regressor)
#     print('\033[1m' + Mtype + ':' + '\033[0m')


    if not Mtype == "LinearRegression":
        
       
        # Add parameters of your choice here:
        if Mtype in("GBTRegressor"):
            paramGrid = (ParamGridBuilder() \
#                          .addGrid(regressor.maxDepth, [2, 5, 10, 20, 30]) \
                         .addGrid(regressor.maxBins, [10, 20]) \
                         .addGrid(regressor.maxIter, [10, 15])
                         .build())

  

        #Cross Validator requires all of the following parameters:
        crossval = CrossValidator(estimator=regressor,
                                  estimatorParamMaps=paramGrid,
                                  evaluator=RegressionEvaluator(metricName="rmse"),
                                  numFolds=2) # 3 is best practice
        # Fit Model: Run cross-validation, and choose the best set of parameters.
        fitModel = crossval.fit(train)
        
        # Get Best Model
        BestModel = fitModel.bestModel

        # FEATURE IMPORTANCES
        # Estimate of the importance of each feature.
        # Each feature’s importance is the average of its importance across all trees 
        # in the ensemble The importance vector is normalized to sum to 1. 
        print(" ")
        print('\033[1m' + Mtype," Feature Importances"+ '\033[0m')
        print("(Scores add up to 1)")
        print("Lowest score is the least important")
        print(" ")
        featureImportances = BestModel.featureImportances.toArray()
        # Convert from numpy array to list
        imp_scores = []
        for x in featureImportances:
            imp_scores.append(float(x))
        # Then zip with input_columns list and create a df
        result = spark.createDataFrame(zip(input_columns,imp_scores), schema=['feature','score'])
        print(result.orderBy(result["score"].desc()).show(truncate=False))
        
        #Create Global Variables for feature importances and models

        if Mtype in("GBTRegressor"):
            global GBT_featureImportances
            GBT_featureImportances = BestModel.featureImportances.toArray()
            global GBT_BestModel 
            GBT_BestModel = fitModel.bestModel

                    
        # Set the column names to match the external results dataframe that we will join with later:
        columns = ['Regressor', 'Result']
        
        # Make predictions.
        predictions = fitModel.transform(test)
        # Select (prediction, true label) and compute test error
        evaluator = RegressionEvaluator(metricName="rmse")
        rmse = evaluator.evaluate(predictions)
        rmse_str = [str(rmse)] #make this a string and convert to a list
        Mtype = [Mtype] #make this a string
        result = spark.createDataFrame(zip(Mtype,rmse_str), schema=columns)
        # Clean up the Result column and output
        result = result.withColumn('Result',result.Result.substr(0, 5))
        return result

### Decision Tree Regressor

In [ ]:
def DecisionTreeRegressormodel(dataframe,input_columns,dependent_var,trainesplite,testsplite,treat_outliers=True):
    """ return  Decision Tree Regressor accuracy 
             
        Parameters:
        -------------- 
            dataframe  (:class:`DataFrame):main dataframe

            dependent_var(str) : dependent varvariable column  

            input_columns(list)  : name of input columns 
            
            train(int) : train splite  
            
            test(int) :   test splite 

            treat_outliers(bool) : treating outliers
                       
        Returns:
        --------------
            result (:class:`DataFrame) : return Decision Tree Regressor accuracy  
              
            """
    final_data=MLRegressDFPrep(dataframe,input_columns,dependent_var,treat_outliers)
    train,test = final_data.randomSplit([trainesplite,testsplite])
    regressor=DecisionTreeRegressor()
    def FindMtype(regressor):
        # Intstantiate Model
        M = regressor
        # Learn what it is
        Mtype = type(M).__name__
        
        return Mtype
    
    Mtype = FindMtype(regressor)
#     print('\033[1m' + Mtype + ':' + '\033[0m')


    if not Mtype == "LinearRegression":
        
       

        # Add parameters of your choice here:
        if Mtype in("DecisionTreeRegressor"):
            paramGrid = (ParamGridBuilder() \
#                          .addGrid(regressor.maxDepth, [2, 5, 10, 20, 30]) \
                         .addGrid(regressor.maxBins, [10, 20, 40]) \
                         .build())

        #Cross Validator requires all of the following parameters:
        crossval = CrossValidator(estimator=regressor,
                                  estimatorParamMaps=paramGrid,
                                  evaluator=RegressionEvaluator(metricName="rmse"),
                                  numFolds=2) # 3 is best practice
        # Fit Model: Run cross-validation, and choose the best set of parameters.
        fitModel = crossval.fit(train)
        
        # Get Best Model
        BestModel = fitModel.bestModel

        # FEATURE IMPORTANCES
        # Estimate of the importance of each feature.
        # Each feature’s importance is the average of its importance across all trees 
        # in the ensemble The importance vector is normalized to sum to 1. 
        print(" ")
        print('\033[1m' + Mtype," Feature Importances"+ '\033[0m')
        print("(Scores add up to 1)")
        print("Lowest score is the least important")
        print(" ")
        featureImportances = BestModel.featureImportances.toArray()
        # Convert from numpy array to list
        imp_scores = []
        for x in featureImportances:
            imp_scores.append(float(x))
        # Then zip with input_columns list and create a df
        result = spark.createDataFrame(zip(input_columns,imp_scores), schema=['feature','score'])
        print(result.orderBy(result["score"].desc()).show(truncate=False))
        
        #Create Global Variables for feature importances and models
        if Mtype in("DecisionTreeRegressor"):
            global DT_featureImportances
            DT_featureImportances = BestModel.featureImportances.toArray()
            global DT_BestModel 
            DT_BestModel = fitModel.bestModel

                    
        # Set the column names to match the external results dataframe that we will join with later:
        columns = ['Regressor', 'Result']
        
        # Make predictions.
        predictions = fitModel.transform(test)
        # Select (prediction, true label) and compute test error
        evaluator = RegressionEvaluator(metricName="rmse")
        rmse = evaluator.evaluate(predictions)
        rmse_str = [str(rmse)] #make this a string and convert to a list
        Mtype = [Mtype] #make this a string
        result = spark.createDataFrame(zip(Mtype,rmse_str), schema=columns)
        # Clean up the Result column and output
        result = result.withColumn('Result',result.Result.substr(0, 5))
        return result

### run  all regression models

In [ ]:
def runregression(regressor,dataframe,input_columns,dependent_var,trainesplite,testsplite,treat_outliers=True):
  """ run  all regression models
             
        Parameters:
        -------------- 
            regressor :select regression models include ()

            dataframe  (:class:`DataFrame):main dataframe

            input_columns(list)  : name of input columns 

            dependent_var(str) : dependent varvariable column  
            
            train(int) : train splite  
            
            test(int) :   test splite 

            treat_outliers(bool) : treating outliers
            
                       
        Returns:
        --------------
            result (:class:`DataFrame) : return all acaccuracy  regression models
              
            """
  def FindMtype(regressor):
        # Intstantiate Model
        M = regressor
        # Learn what it is
        Mtype = type(M).__name__
        
        return Mtype
    
  Mtype = FindMtype(regressor)    
  #set up your results table
  columns = ['Regressor', 'Result']
  vals = [("Place Holder","N/A")]
  results = spark.createDataFrame(vals, columns)
  if Mtype == "LinearRegression":
    new_result = LinearRegressionmodel(dataframe,input_columns,dependent_var,trainesplite,testsplite,treat_outliers)
    results = results.union(new_result)
    results = results.where("Regressor!='Place Holder'")
  elif Mtype == "RandomForestRegressor":
    new_result = RandomForestRegressormodel(dataframe,input_columns,dependent_var,trainesplite,testsplite,treat_outliers)
    results = results.union(new_result)
    results = results.where("Regressor!='Place Holder'")

  elif Mtype in("GBTRegressor"):
    new_result = GBTRegressormodel(dataframe,input_columns,dependent_var,trainesplite,testsplite,treat_outliers)
    results = results.union(new_result)
    results = results.where("Regressor!='Place Holder'")
  elif Mtype in("DecisionTreeRegressor"):  
    new_result = DecisionTreeRegressormodel(dataframe,input_columns,dependent_var,trainesplite,testsplite,treat_outliers)
    results = results.union(new_result)
    results = results.where("Regressor!='Place Holder'")
  return results

### The best regressor model has the lowest root mean squared error (rmse)

In [ ]:
def prediction(bestmodel,dataframe,input_columns:list,dependent_var:str,treat_outliers=True)->tuple:

  """ return tuple with predictions and percentage of diffrence between actual data and predition 
             
        Parameters:
        -------------- 
            bestmodel :select regression models include ()

            dataframe  (:class:`DataFrame):main dataframe

            input_columns(list)  : name of input columns 

            dependent_var(str) : dependent varvariable column  


            treat_outliers(bool) : treating outliers
            
                       
        Returns:
        --------------
            perdiction_df (tuple) : tuple with two dataframes: (predictions) and (percentage of diffrence between actual data and predition )
              
            """
  def FindMtype(bestmodel):
        # Intstantiate Model
        M = bestmodel
        # Learn what it is
        Mtype = type(M).__name__
        
        return Mtype
    
  Mtype = FindMtype(bestmodel)
  final_data=MLRegressDFPrep(dataframe,input_columns,dependent_var,treat_outliers)  
  test = final_data.limit(10)
  if Mtype == "LinearRegression":
      predictions = LR_BestModel.transform(test) 
  if Mtype in("DecisionTreeRegressor"):
      predictions = DT_BestModel.transform(test)
  if Mtype in("GBTRegressor"):
      predictions = GBT_BestModel.transform(test)
  if Mtype in("RandomForestRegressor"):
    predictions = RF_BestModel.transform(test)

  predictions = predictions.withColumn("difference",predictions.prediction-predictions.label) \
                          .withColumn("diff perct",((predictions.prediction-predictions.label)/predictions.label)*100)
  diff_perct=predictions.describe(['diff perct'])
  perdiction_df=predictions,diff_perct
  return perdiction_df
 

In [ ]:
def featureimportances(n:int,model)->list:
    """ return feature importances list
             
        Parameters:
        -------------- 

            n(int) : number of input value
            
            model : select model 
            
                       
        Returns:
        --------------
            feature importances (list) : return feature importances list
              
            """
    def FindMtype(model):
        # Intstantiate Model
        M = model
        # Learn what it is
        Mtype = type(M).__name__
        
        
        return Mtype
    Mtype = FindMtype(model)
    if Mtype == "LinearRegression":
        featureimportances=LR_Pvalues
    if Mtype in("DecisionTreeRegressor"):
        featureimportances=DT_featureImportances.argsort()[-n:][::-1]
    if Mtype in("GBTRegressor"):
        featureimportances=GBT_featureImportances.argsort()[-n:][::-1]
    if Mtype in("RandomForestRegressor"):
        featureimportances=RF_featureImportances.argsort()[-n:][::-1]
   
    
    return featureimportances

In [ ]:
def testmodel(bestmodel,values:list,input_columns:list,skewtreat:bool=False,skewtreatcolumns:list=None): 
  """ return a predition for actual list of input 
             
        Parameters:
        -------------- 
            bestmodel :select regression models include ()

            values(list) : list of input number


            input_columns(list)  : name of input columns 


            skewtreat(bool) : treat for skew columns  


            skewtreatcolumns(list) : list of column need fo skew
            
                       
        Returns:
        --------------
            perdictions (:class:`DataFrame) : return prediction dataframe 
              
            """
 
  def FindMtype(bestmodel):
        # Intstantiate Model
        M = bestmodel
        # Learn what it is
        Mtype = type(M).__name__  
        return Mtype
  Mtype = FindMtype(bestmodel)
  column_names = input_columns
  test = spark.createDataFrame(values,column_names)
  if skewtreat:
      for skewcol in skewtreatcolumns:   
        test = test.withColumn(skewcol, log(skewcol) +1)

  # Transform for a vector
  features_list = input_columns
  assembler = VectorAssembler(inputCols=features_list,outputCol='features')
  test = assembler.transform(test).select('features')
      
  # rescale each feature to range [min, max].
  scaled_test = scalerModel.transform(test)
  final_test = scaled_test.select('scaledFeatures')
  final_test = final_test.withColumnRenamed('scaledFeatures','features')
  if Mtype == "LinearRegression":
      predictions = LR_BestModel.transform(final_test) 
  if Mtype in("DecisionTreeRegressor"):
      predictions = DT_BestModel.transform(final_test)
  if Mtype in("GBTRegressor"):
      predictions = GBT_BestModel.transform(final_test)
  if Mtype in("RandomForestRegressor"):
    predictions = RF_BestModel.transform(final_test)
  return predictions


## Frequent Pattern Mining in PySpark's MLlib

In [ ]:
def FPM(itemsCol:str,p_types,minSupport:int,minConfidence:int,maxPatternLength:int,df_array)->tuple:
  """ return  itempopularity,assoc,predict,sequence dataframes
             
        Parameters:
        --------------
            
            itemsCol(str) : p_types alias column
                
            p_types(:class:`DataFrame) : DataFrame with  unique values in each row

            minSupport(int) : minimum of support        
            
            minConfidence(int) : minimum of Confidence 

            maxPatternLength(int) : maximum Pattern Length
            
            df_array(:class:`DataFrame) : DataFrame with  array of array

 
            
            
                       
        Returns:
        --------------
            silhcent (list) : return a tuple with silhouette,center of kmeans ,prediction dataframe 

        Example:
        --------------  

            p_types = df.withColumn("vert",expr("CASE WHEN EXT1 in('4','5') or EXT5 in('4','5') or EXT7 in('4','5') or EXT9 in('4','5') THEN 'extrovert' WHEN EXT1 in('1','2') or EXT5 in('1','2') or EXT7 in('1','2') or EXT9 in('1','2') THEN 'introvert' ELSE 'neutrovert' END AS vert"))
            p_types = p_types.withColumn("mood",expr("CASE WHEN EST2 in('4','5') THEN 'chill' WHEN EST2 in('1','2') THEN 'highstrung' ELSE 'neutral' END AS mood"))

            p_types = p_types.select(array('mood', 'vert').alias("items"))
            p_types.limit(4).toPandas()

            df_array = df.select(array(array('EXT1', 'EXT2'),array('EXT3','EXT4'),array('EXT5','EXT6'),array('EXT7','EXT8'),array('EXT9','EXT10')).alias("sequence"))

           """ 
  from pyspark.ml.fpm import FPGrowth
  from pyspark.ml.fpm import PrefixSpan
  fpGrowth = FPGrowth(itemsCol=itemsCol, minSupport=minSupport, minConfidence=minConfidence)
  model = fpGrowth.fit(p_types)
  itempopularity = model.freqItemsets

  assoc = model.associationRules


  predict = model.transform(p_types)


  prefixSpan = PrefixSpan(minSupport=minSupport, maxPatternLength=maxPatternLength)

  sequence = prefixSpan.findFrequentSequentialPatterns(df_array)
  rolebase=itempopularity,assoc,predict,sequence
  return rolebase


In [ ]:
def sequence(itemsCol:str,p_types,minSupport:int,minConfidence:int,maxPatternLength:int,df_array):
  """ return  itempopularity,assoc,predict,sequence dataframes
             
        Parameters:
        --------------
            
            itemsCol(str) : p_types alias column
                
            p_types(:class:`DataFrame) : DataFrame with  unique values in each row

            minSupport(int) : minimum of support        
            
            minConfidence(int) : minimum of Confidence 

            maxPatternLength(int) : maximum Pattern Length
            
            df_array(:class:`DataFrame) : DataFrame with  array of array

 
            
            
                       
        Returns:
        --------------
            filtered(:class:`DataFrame) : DataFrame with  sequence percentage 

        Example:
        --------------  

            p_types = df.withColumn("vert",expr("CASE WHEN EXT1 in('4','5') or EXT5 in('4','5') or EXT7 in('4','5') or EXT9 in('4','5') THEN 'extrovert' WHEN EXT1 in('1','2') or EXT5 in('1','2') or EXT7 in('1','2') or EXT9 in('1','2') THEN 'introvert' ELSE 'neutrovert' END AS vert"))
            p_types = p_types.withColumn("mood",expr("CASE WHEN EST2 in('4','5') THEN 'chill' WHEN EST2 in('1','2') THEN 'highstrung' ELSE 'neutral' END AS mood"))

            p_types = p_types.select(array('mood', 'vert').alias("items"))
            p_types.limit(4).toPandas()

            df_array = df.select(array(array('EXT1', 'EXT2'),array('EXT3','EXT4'),array('EXT5','EXT6'),array('EXT7','EXT8'),array('EXT9','EXT10')).alias("sequence"))

           """ 
  from pyspark.sql.functions import expr, round
  sequence=FPM(itemsCol,p_types,minSupport,minConfidence,maxPatternLength,df_array)

  #get the size of each array within the arrays
  filtered = sequence.withColumn('size', expr('transform(sequence, x -> size(x))'))
  # sequence.withColumn("length",size(col("sequence")))

  # Let's also add a column that tells us the percentage of the sequences
  row_cnt = df_array.count()
  filtered = filtered.withColumn('percentage',round((col("freq")/row_cnt)*100,2))
  # Then filter out only the ones with more than 2 elements
  filtered = filtered.where(array_contains(filtered.size, 2))

  return filtered
